## First Attempt of Shear Profile of A360 with metadetection

Contact author: Miranda Gorsuch

First attempt at creating a shear profile for A360 using cell-based coadds and `metadetection`. Many parts are from the [Shear profile around A360 using ComCam HSM shapes](https://github.com/lsst-sitcom/comcam_clusters/blob/main/ACO360_WL_HSCcalib_CLMM.ipynb) notebook, especially the identification of red sequence galaxies and use of CLMM to create the tangential shear plot.

Last working weekly: `w_2025_26`

Container Size: small (4 GB) for shear profile, may need 8 GB for some validation plots

# Preparing data

## Imports & Definitions

In [ ]:
# locally install modeling packages (only do once, if not already installed)
# pip install pyccl
# pip install clmm

In [ ]:
from lsst.daf.butler import Butler

import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
import matplotlib

import astropy.units as u
from astropy.table import Table
from astropy.coordinates import SkyCoord

import gc

from lsst.skymap import Index2D
import lsst.afw.geom as afwGeom
import lsst.afw.math as afwMath
import lsst.afw.image as afwImage
import lsst.geom as geom
import lsst.afw.display.rgb as afwRgb
import treecorr

# for stats control
from lsst.drp.tasks.assemble_cell_coadd import AssembleCellCoaddTask
import lsst.meas.algorithms as meas

from lsst.afw.geom.ellipses import Quadrupole, SeparableDistortionTraceRadius

from numpy.linalg import inv
import scipy.stats as stats

import clmm
from clmm import GalaxyCluster, ClusterEnsemble, GCData, Cosmology
from clmm import Cosmology, utils

cosmo = clmm.Cosmology(H0=70.0, Omega_dm0=0.3 - 0.045, Omega_b0=0.045, Omega_k0=0.0)

%matplotlib inline

In [ ]:
REPO = '/sdf/data/rubin/repo/main/'
butler = Butler(REPO)
registry = butler.registry

collection = 'u/mgorsuch/metadetect/a360_3_band/20250612T151357Z' # metadetect run on g, r, i
cell_collection = 'u/mgorsuch/a360_cell_coadd/20250513T044026Z' # cells used as input for r and i
cell_collection_g = 'u/mgorsuch/a360_cell_coadd_g/20250611T144112Z' # cells used as input for g

In [ ]:
# create and configure stats control object as seen in assemble_cell_coadd task
statsCtrl = afwMath.StatisticsControl()
statsCtrl.setAndMask(afwImage.Mask.getPlaneBitMask(("BAD", "NO_DATA", "SAT"))) # use default PlaneBitMasks from task
statsCtrl.setNanSafe(True)

## Read in data

Metadetect outputs tables for each patch. Read in each table and compile them together.

In [ ]:
# Position in degrees of the BCG for A360
ra_bcg = 37.865017
dec_bcg = 6.982205

The cell below is for finding the tracts/patches that are within the specified radius of the BCG. This is already incorporated in the butler collection used for the `metadetect` output.

In [ ]:
# skymap = butler.get('skyMap', skymap='lsst_cells_v1', collections='LSSTComCam/runs/DRP/DP1/w_2025_06/DM-48810')

# # Looking for all patches in delta deg region around it
# delta = 0.5
# center = geom.SpherePoint(ra_bcg, dec_bcg, geom.degrees)
# ra_min, ra_max = ra_bcg - delta, ra_bcg + delta
# dec_min, dec_max = dec_bcg - delta, dec_bcg + delta

# ra_range = (ra_min, ra_max)
# dec_range = (dec_min, dec_max)
# radec = [geom.SpherePoint(ra_range[0], dec_range[0], geom.degrees),
#          geom.SpherePoint(ra_range[0], dec_range[1], geom.degrees),
#          geom.SpherePoint(ra_range[1], dec_range[0], geom.degrees),
#          geom.SpherePoint(ra_range[1], dec_range[1], geom.degrees)]

# tractPatchList = skymap.findTractPatchList(radec)

# find dataset refs that are within the tract/patch list above
# datasetRefs_shear = []
# datasetRefs_catalog = []

# for tractPatch in tractPatchList:
#     tract = tractPatch[0]
#     patchInfo = tractPatch[1]
#     for patch in patchInfo:
#         datasetRefs_shear.append(butler.query_datasets('ShearObject', 
#                                                  collections=collection,
#                                                  tract=tract.tract_id,
#                                                  patch=patch.sequential_index))
#         datasetRefs_catalog.append(butler.query_datasets('objectTable', 
#                                                  collections=default_collection,
#                                                  tract=tract.tract_id,
#                                                  patch=patch.sequential_index))

In [ ]:
datasetRefs_shear = []
overlap_patches_10463 = [0, 10, 20, 30, 40, 50, 60, 70, 80, 90, 100]
tract_patch_list = [] # only used for plotting distribution of input images

for ref in butler.registry.queryDatasets('ShearObject', collections=collection):

    # the only parts of these tracts within 0.5 radius overlap with already included patches
    if ref.dataId['tract'] == 10704 or ref.dataId['tract'] == 10705:
        continue

    # these column of patches overlap with patches already in tract 10464
    if ref.dataId['tract'] == 10463 and ref.dataId['patch'] in overlap_patches_10463:
        continue
    
    datasetRefs_shear.append(butler.query_datasets('ShearObject', 
                                                     collections=collection,
                                                     skymap = 'lsst_cells_v1',
                                                     tract=ref.dataId['tract'],
                                                     patch=ref.dataId['patch']))

    tract_patch_list.append([ref.dataId['tract'], ref.dataId['patch']])

Combine the data from each patch into a single table.

In [ ]:
shear_table_list = []

for i, ref in enumerate(datasetRefs_shear):
    shear_data_patch = butler.get(ref[0])
    shear_table_patch = shear_data_patch.to_pandas()
    shear_table_list.append(shear_table_patch)

shear_table = pd.concat(shear_table_list)

# remove unused tables to clear up memory
del shear_table_list
gc.collect()

### Remove duplicate objects from patch overlap

In [ ]:
# copy table prior to duplicate removal for a later validation plot
shear_table_dup = shear_table.copy()

In [ ]:
# remove objects in outer ring of cells in each patch since patch overlap is two cells
# TO-DO: exempt rows that don't overlap with other patches, e.g. patches on the edge of the field
shear_table = shear_table[shear_table['cell_x']!=0]
shear_table = shear_table[shear_table['cell_x']!=21]
shear_table = shear_table[shear_table['cell_y']!=0]
shear_table = shear_table[shear_table['cell_y']!=21]
print("Number of rows after removing most duplicate cells: ", len(shear_table))

# some additional tract/patch overlap appears to have a 4 cell overlap
filt1 = shear_table['tract'] == 10464
filt1 &= shear_table['patch_x'] == 9
filt1 &= shear_table['cell_x'] == 20
shear_table = shear_table[np.invert(filt1)]

filt2 = shear_table['tract'] == 10463
filt2 &= shear_table['patch_x'] == 1
filt2 &= shear_table['cell_x'] == 1
shear_table = shear_table[np.invert(filt2)]
print("Number of rows after removing patch overlap in 10464: ", len(shear_table))

In [ ]:
# remove overlapping rows due to patch overlap    
print("Number of rows prior to removing duplicates: ", len(shear_table))
shear_table = shear_table.drop_duplicates(subset=['shear_type', 'ra', 'dec']) # each object will potentially have several sheared images
print("Number of rows after removing duplicates: ", len(shear_table))

### Add useful columns

In [ ]:
# make new columns to convert nJy fluxes to AB magnitudes
t1 = Table.from_pandas(shear_table)

t1['wmom_band_mag_g'] = (t1['wmom_band_flux_g']*u.nJy).to(u.ABmag)
t1['wmom_band_mag_r'] = (t1['wmom_band_flux_r']*u.nJy).to(u.ABmag)
t1['wmom_band_mag_i'] = (t1['wmom_band_flux_i']*u.nJy).to(u.ABmag)
t1['wmom_color_mag_g-r'] = t1['wmom_band_mag_g']-t1['wmom_band_mag_r']
t1['wmom_color_mag_g-i'] = t1['wmom_band_mag_g']-t1['wmom_band_mag_i']
t1['wmom_color_mag_r-i'] = t1['wmom_band_mag_r']-t1['wmom_band_mag_i']

shear_table = t1.to_pandas()

# Add columns for distance from BCG
c1 = SkyCoord(shear_table['ra'].values*u.deg, shear_table['dec'].values*u.deg)
c2 = SkyCoord(ra_bcg*u.deg, dec_bcg*u.deg)
sep = c1.separation(c2)

shear_table['deg_sep'] = sep.value

shear_table['mpc_sep'] = cosmo.eval_da(0.22) * shear_table['deg_sep'] * np.pi/180

# polar angle of source galaxy relative to BCG, from -pi to pi
shear_table['phi'] = np.arctan2(shear_table['dec'] - dec_bcg, (ra_bcg - shear_table['ra'])*np.cos(np.deg2rad(dec_bcg)))

# calculate tangential / cross shear components for each galaxy
# note that these shears need to be selected for ONLY the 'ns' (no artificial shear type)
shear_table['shear_t'] = -shear_table['wmom_g_1'] * np.cos(2*shear_table['phi']) \
                                - shear_table['wmom_g_2'] * np.sin(2*shear_table['phi'])
shear_table['shear_x'] = shear_table['wmom_g_1'] * np.sin(2*shear_table['phi']) \
                                - shear_table['wmom_g_2'] * np.cos(2*shear_table['phi'])

## Apply `metadetect` flags

Anything that is flagged should be removed.

In [ ]:
meta_filter = shear_table['wmom_flags']==False
meta_filter &= shear_table['psfrec_flags']==False
meta_filter &= shear_table['wmom_psf_flags']==False
meta_filter &= shear_table['wmom_obj_flags']==False
meta_filter &= shear_table['wmom_T_flags']==False
meta_filter &= shear_table['wmom_band_flux_flags_r']==False
meta_filter &= shear_table['wmom_band_flux_flags_i']==False

shear_table = shear_table[meta_filter]

print("Number of rows after removing metadetect flags: ", len(shear_table))
print("Number of rows in ns after removing metadetect flags: ", len(shear_table[shear_table['shear_type']=='ns']))

shear_table_md_cuts = shear_table.copy()

## Identify and remove cluster member galaxies

In [ ]:
# plotting parameters
point_size = 0.8
point_alpha = 0.7

r_left = 19
r_right = 23
r_down_left = 0.4
r_up_left = 0.6
r_down_right = 0.3
r_up_right = 0.5
r_slope = (r_down_right - r_down_left) / (r_right - r_left)

g_left = 21.5
g_right = 23
g_down_left = 1.7
g_up_left = 2
g_down_right = 1.4
g_up_right = 1.7
g_slope = (g_down_right - g_down_left) / (g_right - g_left)

brightness_cut = 23

### RS Filter (one cell)

For applying RS cuts without running all of the visual inspection cells (though based on those visual inspections).

In [ ]:
shear_table_wl = shear_table[shear_table['deg_sep'] < 0.5] # catalog for WL measurements
print("Number of rows after applying < 0.5 deg from center: ", len(shear_table_wl))

rs_hi_ri = r_up_left + r_slope * (shear_table_wl['wmom_band_mag_r']-r_left)
rs_low_ri = r_down_left + r_slope * (shear_table_wl['wmom_band_mag_r']-r_left)
ri_filt = shear_table_wl['wmom_color_mag_r-i'] > rs_low_ri
ri_filt &= shear_table_wl['wmom_color_mag_r-i'] < rs_hi_ri
ri_filt &= shear_table_wl['wmom_band_mag_r'] < brightness_cut

rs_hi_gi = g_up_left + g_slope * (shear_table_wl['wmom_band_mag_g']-g_left)
rs_low_gi = g_down_left + g_slope * (shear_table_wl['wmom_band_mag_g']-g_left)
gi_filt = shear_table_wl['wmom_color_mag_g-i'] > rs_low_gi
gi_filt &= shear_table_wl['wmom_color_mag_g-i'] < rs_hi_gi
gi_filt &= shear_table_wl['wmom_band_mag_g'] < brightness_cut

# ns only, for plotting
ri_filt_ns = np.logical_and(ri_filt, shear_table_wl['shear_type'] == 'ns')
gi_filt_ns = np.logical_and(gi_filt, shear_table_wl['shear_type'] == 'ns')

rs_filt = np.logical_and(ri_filt, gi_filt)
rs_filt_ns = np.logical_and(rs_filt, shear_table_wl['shear_type'] == 'ns')

RS_id_list = shear_table_wl['id'][rs_filt]
RS_id_list_ns = shear_table_wl_ns['id'][rs_filt_ns] # for plotting

# Filter out rows where the 'dataid' column matches any value in RS_id_list
shear_table_wl = shear_table_wl[~shear_table_wl['id'].isin(RS_id_list)]
shear_table_wl_ns = shear_table_wl[shear_table_wl['shear_type']=='ns']

print("Number of rows after 0.5 degree cut and RS cuts: ", len(shear_table_wl))
print("Number of rows after 0.5 degree cut and RS cuts, ns only: ", len(shear_table_wl_ns))

shear_table_rs_cuts = shear_table_wl.copy()

### Visual Inspections

Cuts are applied to all shear type catalogs, though only non-sheared are plotted.

In [ ]:
# isolate no shear catalog
shear_table_ns = shear_table[shear_table['shear_type']=='ns']

# distribution of magnitudes prior to cuts
fig, ax = plt.subplots(nrows=1, ncols=3, figsize=(12,4))
ax[0].hist(shear_table_ns['wmom_band_mag_g'], bins=75)
ax[0].set_title("g-band")
ax[0].vlines(x=24.7, ymin=0, ymax=14000, color='red')
ax[1].hist(shear_table_ns['wmom_band_mag_r'], bins=75)
ax[1].set_title("r-band")
ax[1].vlines(x=24.2, ymin=0, ymax=14000, color='red')
ax[2].hist(shear_table_ns['wmom_band_mag_i'], bins=75)
ax[2].set_title("i-band")
ax[2].vlines(x=23.6, ymin=0, ymax=14000, color='red')

for ax in ax.reshape(-1):
    ax.set_yscale('log')
    ax.set_xlabel("AB Mag")
    ax.set_xlim(right=27)
    ax.set_ylim(top=14000)

# plt.savefig('image_outputs/object-magnitudes.png', bbox_inches='tight')
plt.suptitle("Object Magnitudes")

#### Color-Magnitude Plots

In [ ]:
# Filter objects to within < 0.1 deg of cluster center
filt = shear_table['deg_sep'] < 0.1 # stay close to cluster center for RS indentification
shear_table_rs = shear_table[shear_table['deg_sep'] < 0.1] # catalog for RS identification
print("Number of rows in after applying < 0.1 deg from center for all shear types: ", len(shear_table_rs))

In [ ]:
shear_table_rs_ns = shear_table_rs[shear_table_rs['shear_type']=='ns']

In [ ]:
fig, axes = plt.subplots(1,2, figsize=(12,5))

axes[0].scatter(shear_table_rs_ns['wmom_band_mag_r'], shear_table_rs_ns['wmom_color_mag_r-i'], 
           marker='.', s=point_size, alpha=point_alpha)
axes[0].set_ylabel('r-i')
axes[0].set_xlabel('r')

axes[0].plot([r_left,r_right],[r_down_left,r_down_right], color='r', linewidth=0.7)
axes[0].plot([r_left,r_right],[r_up_left,r_up_right], color='r', linewidth=0.7)
axes[0].set_ylim([-1.5,2.5])
axes[0].set_xlim([17,24.5])

axes[1].scatter(shear_table_rs_ns['wmom_band_mag_g'], shear_table_rs_ns['wmom_color_mag_g-i'], 
           marker='.', s=point_size, alpha=point_alpha)
axes[1].set_ylabel('g-i')
axes[1].set_xlabel('g')

axes[1].plot([g_left,g_right],[g_down_left,g_down_right], color='r', linewidth=0.7)
axes[1].plot([g_left,g_right],[g_up_left,g_up_right], color='r', linewidth=0.7)
axes[1].set_ylim([-0.5,3])
axes[1].set_xlim([20,25])

plt.suptitle("Color-Magnitude < 0.1 deg")

# plt.savefig('image_outputs/color-magnitude-0-1.png', bbox_inches='tight')
plt.show()

Note that combining all catalogs causes a "smeary" look is due to the 5 sheared/unsheared images of the same object that are detected & measured in slightly different ways.

#### Filter RS galaxies

In [ ]:
rs_hi_ri = r_up_left + r_slope * (shear_table_rs['wmom_band_mag_r']-r_left)
rs_low_ri = r_down_left + r_slope * (shear_table_rs['wmom_band_mag_r']-r_left)
ri_filt = shear_table_rs['wmom_color_mag_r-i'] > rs_low_ri
ri_filt &= shear_table_rs['wmom_color_mag_r-i'] < rs_hi_ri
ri_filt &= shear_table_rs['wmom_band_mag_r'] < brightness_cut

rs_hi_gi = g_up_left + g_slope * (shear_table_rs['wmom_band_mag_g']-g_left)
rs_low_gi = g_down_left + g_slope * (shear_table_rs['wmom_band_mag_g']-g_left)
gi_filt = shear_table_rs['wmom_color_mag_g-i'] > rs_low_gi
gi_filt &= shear_table_rs['wmom_color_mag_g-i'] < rs_hi_gi
gi_filt &= shear_table_rs['wmom_band_mag_g'] < brightness_cut

# ns only, for plotting
ri_filt_ns = np.logical_and(ri_filt, shear_table_rs['shear_type'] == 'ns')
gi_filt_ns = np.logical_and(gi_filt, shear_table_rs['shear_type'] == 'ns')

In [ ]:
fig, axes = plt.subplots(1,2, figsize=(12,5))

axes[0].scatter(shear_table_rs_ns['wmom_band_mag_r'], shear_table_rs_ns['wmom_color_mag_r-i'], 
           marker='.', s=point_size) # all galaxies  
axes[0].scatter(shear_table_rs['wmom_band_mag_r'][ri_filt_ns], 
           shear_table_rs['wmom_color_mag_r-i'][ri_filt_ns], 
           marker='.', s=point_size) #red sequence galaxies
axes[0].set_ylabel('r-i')
axes[0].set_xlabel('r')

axes[0].plot([r_left,r_right],[r_down_left,r_down_right], color='r', linewidth=0.7)
axes[0].plot([r_left,r_right],[r_up_left,r_up_right], color='r', linewidth=0.7)
axes[0].set_ylim([-1.5,2.5])
axes[0].set_xlim([17,24.5])

axes[1].scatter(shear_table_rs_ns['wmom_band_mag_g'], shear_table_rs_ns['wmom_color_mag_g-i'], 
           marker='.', s=point_size) # all galaxies  
axes[1].scatter(shear_table_rs['wmom_band_mag_g'][gi_filt_ns], 
           shear_table_rs['wmom_color_mag_g-i'][gi_filt_ns], 
           marker='.', s=point_size)
axes[1].set_ylabel('g-i')
axes[1].set_xlabel('g')

axes[1].plot([g_left,g_right],[g_down_left,g_down_right], color='r', linewidth=0.7)
axes[1].plot([g_left,g_right],[g_up_left,g_up_right], color='r', linewidth=0.7)
axes[1].set_ylim([-0.5,3])
axes[1].set_xlim([19,24])

plt.suptitle("Color-Magnitude < 0.1 deg\nCuts highlighted")

# plt.savefig('image_outputs/color-magnitude-0-1-orange.png', bbox_inches='tight')
plt.show()

Cut out the RS galaxies identified above.

In [ ]:
shear_table_wl = shear_table[shear_table['deg_sep'] < 0.5] # catalog for WL measurements
print("Number of rows in ns after applying < 0.5 deg from center: ", len(shear_table_wl))

In [ ]:
rs_hi_ri = r_up_left + r_slope * (shear_table_wl['wmom_band_mag_r']-r_left)
rs_low_ri = r_down_left + r_slope * (shear_table_wl['wmom_band_mag_r']-r_left)
ri_filt = shear_table_wl['wmom_color_mag_r-i'] > rs_low_ri
ri_filt &= shear_table_wl['wmom_color_mag_r-i'] < rs_hi_ri
ri_filt &= shear_table_wl['wmom_band_mag_r'] < brightness_cut

rs_hi_gi = g_up_left + g_slope * (shear_table_wl['wmom_band_mag_g']-g_left)
rs_low_gi = g_down_left + g_slope * (shear_table_wl['wmom_band_mag_g']-g_left)
gi_filt = shear_table_wl['wmom_color_mag_g-i'] > rs_low_gi
gi_filt &= shear_table_wl['wmom_color_mag_g-i'] < rs_hi_gi
gi_filt &= shear_table_wl['wmom_band_mag_g'] < brightness_cut

# ns only, for plotting
ri_filt_ns = np.logical_and(ri_filt, shear_table_wl['shear_type'] == 'ns')
gi_filt_ns = np.logical_and(gi_filt, shear_table_wl['shear_type'] == 'ns')

shear_table_wl_ns = shear_table_wl[shear_table_wl['shear_type'] == 'ns']

In [ ]:
fig, axes = plt.subplots(1,2, figsize=(12,5))

axes[0].scatter(shear_table_wl_ns['wmom_band_mag_r'], shear_table_wl_ns['wmom_color_mag_r-i'], 
           marker='.', s=point_size) # all galaxies  
axes[0].set_ylabel('r-i')
axes[0].set_xlabel('r')

axes[0].set_ylim([-1.5,2.5])
axes[0].set_xlim([17,24.5])

axes[1].scatter(shear_table_wl_ns['wmom_band_mag_g'], shear_table_wl_ns['wmom_color_mag_g-i'], 
           marker='.', s=point_size) 
axes[1].set_ylabel('g-i')
axes[1].set_xlabel('g')

axes[1].set_ylim([-0.5,3.5])
axes[1].set_xlim([19,24.5])

plt.suptitle("Color-Magnitude < 0.5 deg")

# plt.savefig('image_outputs/color-magnitude-0-5-no-line.png', bbox_inches='tight')
plt.show()

In [ ]:
fig, axes = plt.subplots(1,2, figsize=(12,5))

axes[0].scatter(shear_table_wl_ns['wmom_band_mag_r'], shear_table_wl_ns['wmom_color_mag_r-i'], 
           marker='.', s=point_size) # all galaxies  
axes[0].scatter(shear_table_wl['wmom_band_mag_r'][ri_filt_ns], 
           shear_table_wl['wmom_color_mag_r-i'][ri_filt_ns], 
           marker='.', s=point_size) #red sequence galaxies
axes[0].set_ylabel('r-i')
axes[0].set_xlabel('r')

axes[0].plot([r_left,r_right],[r_down_left,r_down_right], color='r', linewidth=0.7)
axes[0].plot([r_left,r_right],[r_up_left,r_up_right], color='r', linewidth=0.7)
axes[0].set_ylim([-1.5,2.5])
axes[0].set_xlim([17,24.5])

axes[1].scatter(shear_table_wl_ns['wmom_band_mag_g'], shear_table_wl_ns['wmom_color_mag_g-i'], 
           marker='.', s=point_size) # all galaxies  
axes[1].scatter(shear_table_wl['wmom_band_mag_g'][gi_filt_ns], 
           shear_table_wl['wmom_color_mag_g-i'][gi_filt_ns], 
           marker='.', s=point_size)
axes[1].set_ylabel('g-i')
axes[1].set_xlabel('g')

axes[1].plot([g_left,g_right],[g_down_left,g_down_right], color='r', linewidth=0.7)
axes[1].plot([g_left,g_right],[g_up_left,g_up_right], color='r', linewidth=0.7)
axes[1].set_ylim([-0.5,3])
axes[1].set_xlim([19,24])

plt.suptitle("Color-Magnitude < 0.5 deg\nCuts highlighted")

# plt.savefig('image_outputs/color-magnitude-0-5-orange.png', bbox_inches='tight')
plt.show()

In [ ]:
rs_filt = np.logical_and(ri_filt, gi_filt)
rs_filt_ns = np.logical_and(rs_filt, shear_table_wl['shear_type'] == 'ns')

RS_id_list = shear_table_wl['id'][rs_filt]
RS_id_list_ns = shear_table_wl_ns['id'][rs_filt_ns] # for plotting

In [ ]:
print(len(RS_id_list))
print(len(RS_id_list_ns)) # should be ~20% of above line

In [ ]:
# Filter out rows where the 'dataid' column matches any value in RS_id_list
shear_table_wl = shear_table_wl[~shear_table_wl['id'].isin(RS_id_list)]
shear_table_wl_ns = shear_table_wl_ns[~shear_table_wl_ns['id'].isin(RS_id_list_ns)]

In [ ]:
print(len(shear_table_wl))
print(len(shear_table_wl_ns))

In [ ]:
fig, axes = plt.subplots(1,2, figsize=(12,5))

axes[0].scatter(shear_table_wl_ns['wmom_band_mag_r'], shear_table_wl_ns['wmom_color_mag_r-i'], 
           marker='.', s=point_size) # all galaxies  
axes[0].set_ylabel('r-i')
axes[0].set_xlabel('r')

axes[0].set_ylim([-1.5,2.5])
axes[0].set_xlim([17,24.5])

axes[1].scatter(shear_table_wl_ns['wmom_band_mag_g'], shear_table_wl_ns['wmom_color_mag_g-i'], 
           marker='.', s=point_size) 
axes[1].set_ylabel('g-i')
axes[1].set_xlabel('g')

axes[1].set_ylim([-0.5,3.5])
axes[1].set_xlim([19,24.5])

plt.suptitle("Color-Magnitude < 0.5 deg\nCuts removed")

# plt.savefig('image_outputs/color-magnitude-0-5-removed.png', bbox_inches='tight')
plt.show()

### Check lines of overdensity

Lines of overdensity: imperfect overlap of objects between patches/tracts.

(Outdated, but code might be useful later)

In [ ]:
# patch_list = []

# for ref in butler.registry.queryDatasets('deepCoaddCell', collections=cell_collection, band='i'):
#     patch_list.append(butler.query_datasets('deepCoaddCell', 
#                                                  collections=cell_collection,
#                                                  skymap = 'lsst_cells_v1',
#                                                  band = 'i',
#                                                  tract=ref.dataId['tract'],
#                                                  patch=ref.dataId['patch'])[0])

In [ ]:
# segs = []

# for ref in patch_list:
    
#     coadd = butler.get('deepCoaddCell',
#                       collections=cell_collection,
#                       skymap = 'lsst_cells_v1',
#                       band = 'i',
#                       tract=ref.dataId['tract'],
#                       patch=ref.dataId['patch'])

#     wcs = coadd.wcs
#     bbox = coadd.inner_bbox

#     coadd_corners = coadd.inner_bbox.getCorners()

#     for index, corner in enumerate(coadd_corners):
#         corner_coord_start = wcs.pixelToSky(corner.getX(), corner.getY())
#         if index < 3:
#             corner_coord_end = wcs.pixelToSky(coadd_corners[index+1].getX(), coadd_corners[index+1].getY())
#         else:
#             corner_coord_end = wcs.pixelToSky(coadd_corners[0].getX(), coadd_corners[0].getY())
    
#         start_ra = corner_coord_start[0].asDegrees()
#         start_dec = corner_coord_start[1].asDegrees()
    
#         end_ra = corner_coord_end[0].asDegrees()
#         end_dec = corner_coord_end[1].asDegrees()

#         segs.append(((start_ra, start_dec), (end_ra, end_dec)))

#     del coadd
#     gc.collect()

In [ ]:
# plt.scatter(ra, dec, marker='.', s=0.2)
# plt.scatter([ra_bcg], [dec_bcg], marker='+', s=100, color='orange')
# for seg in segs:
#     plt.plot([seg[0][0], seg[1][0]], [seg[0][1], seg[1][1]], 'r-', alpha=0.4)
# plt.show()

## Quality cuts

Using cuts from [Yamamoto 2024](https://arxiv.org/abs/2501.05665) for
- Star/galaxy separation
- S/N
- Size (visually check first if needed)
- Flux
- Color

These may need to be updated provided this is a different dataset, but should be a good starting point.

**Note: the Yamamoto paper cut for object size is definited differently than the Metadetect columns, so the object size cut from the DES code is used instead https://github.com/des-science/des-y6utils/blob/main/des_y6utils/mdet.py#L268**

In [ ]:
def print_rows_removed(og_table, cuts_name):
    total = len(og_table)
    print("Cut name: ", cuts_name)
    if cuts_name=="final_cuts":
        print("wmom_T_ratio rows removed: ", total-len(og_table[og_table['wmom_T_ratio']>1.1]), "  (", \
            (1 - len(og_table[og_table['wmom_T_ratio']>1.1]) / total))
        print("wmom_s2n rows removed: ", total-len(og_table[og_table['wmom_s2n']>10]), "  (", \
            (1 - len(og_table[og_table['wmom_s2n']>10]) / total))
        print("wmom_T rows removed: ", total-len(og_table[og_table['wmom_T']<20]), "  (", \
            (1 - len(og_table[og_table['wmom_T']<20]) / total))
        print("m_frac rows removed: ", total-len(og_table[og_table['mfrac']<0.1]), "  (", \
            (1 - len(og_table[og_table['mfrac']<0.1]) / total))
        print("wmom_band_mag_g rows removed: ", total-len(og_table[og_table['wmom_band_mag_g']<24.7]), "  (", \
            (1 - len(og_table[og_table['wmom_band_mag_g']<24.7]) / total))
        print("wmom_band_mag_r rows removed: ", total-len(og_table[og_table['wmom_band_mag_r']<24.2]), "  (", \
            (1 - len(og_table[og_table['wmom_band_mag_r']<24.2]) / total))
        print("wmom_band_mag_i rows removed: ", total-len(og_table[og_table['wmom_band_mag_i']<23.6]), "  (", \
            (1 - len(og_table[og_table['wmom_band_mag_i']<23.6]) / total))
        print("wmom_color_mag_g-r rows removed: ", total-len(og_table[(og_table['wmom_color_mag_g-r']).abs()<5]), "  (", \
            (1 - len(og_table[(og_table['wmom_color_mag_g-r']).abs()<5]) / total))
        print("wmom_color_mag_r-i rows removed: ", total-len(og_table[(og_table['wmom_color_mag_r-i']).abs()<5]), "  (", \
            (1 - len(og_table[(og_table['wmom_color_mag_r-i']).abs()<5]) / total))
        print("wmom_band_mag_i<20 rows removed: ", total-len(og_table[og_table['wmom_band_mag_i']>20]), "  (", \
            (1 - len(og_table[og_table['wmom_band_mag_i']>20]) / total))
        print("Junk cut 1: ", total-len(og_table[og_table['wmom_T'] < (0.425 - 2.0*og_table['wmom_T_err'])]), "  (", \
            (1 - len(og_table[og_table['wmom_T'] < (0.425 - 2.0*og_table['wmom_T_err'])]) / total))
        print("Junk cut 2: ", total-len(og_table[(og_table['wmom_T'] * og_table['wmom_T_err']) < 0.006]), "  (", \
            (1 - len(og_table[(og_table['wmom_T'] * og_table['wmom_T_err']) < 0.006]) / total))
    else:
        print("Not a valid cut or missing column name")

In [ ]:
# cuts determined mainly from Yamamoto but also additional analyses here
final_cuts = shear_table_wl['wmom_T_ratio']>1.1
final_cuts &= shear_table_wl['wmom_s2n']>10
final_cuts &= shear_table_wl['wmom_T']<20
final_cuts &= shear_table_wl['mfrac']<0.1
final_cuts &= shear_table_wl['wmom_band_mag_g']<24.7
final_cuts &= shear_table_wl['wmom_band_mag_r']<24.2
final_cuts &= shear_table_wl['wmom_band_mag_i']<23.6
final_cuts &= (shear_table_wl['wmom_color_mag_g-r']).abs()<5
final_cuts &= (shear_table_wl['wmom_color_mag_r-i']).abs()<5
final_cuts &= shear_table_wl['wmom_band_mag_i']>20
# junk cuts
final_cuts &= shear_table_wl['wmom_T'] < (0.425 - 2.0*shear_table_wl['wmom_T_err'])
final_cuts &= (shear_table_wl['wmom_T'] * shear_table_wl['wmom_T_err']) < 0.006

cut_name, cut_name_string = final_cuts, "final_cuts"
print_rows_removed(shear_table_wl, cut_name_string)

shear_table_wl = shear_table_wl[cut_name]
shear_table_wl_ns = shear_table_wl[shear_table_wl['shear_type']=='ns']

print()
print("Number of rows after applying quality cuts: ", len(shear_table_wl))

# Looking at shear outputs

## Check shear types for each object

Each object is detected and measured separately for each sheared/unsheared image. The catalogs will not necessarily be the same but should be close in number of objects.

In [ ]:
# split catalog by shear type
shear_table_wl_ns = shear_table_wl[shear_table_wl['shear_type']=='ns']
shear_table_wl_1p = shear_table_wl[shear_table_wl['shear_type']=='1p']
shear_table_wl_1m = shear_table_wl[shear_table_wl['shear_type']=='1m']
shear_table_wl_2p = shear_table_wl[shear_table_wl['shear_type']=='2p']
shear_table_wl_2m = shear_table_wl[shear_table_wl['shear_type']=='2m']

In [ ]:
print("Number of shear type 'ns': ", len(shear_table_wl_ns))
print("Number of shear type '1p': ", len(shear_table_wl_1p))
print("Number of shear type '1m': ", len(shear_table_wl_1m))
print("Number of shear type '2p': ", len(shear_table_wl_2p))
print("Number of shear type '2m': ", len(shear_table_wl_2m))

## Determining tangential & cross shear

In [ ]:
# Radial binning, either in Mpc or degrees
bins_deg = clmm.make_bins(0.025,0.5,nbins=6, method='evenlog10width')
bins_mpc = cosmo.eval_da(0.22) * bins_deg * np.pi/180

# define distance bins
dig_rad_bins_ns = np.digitize(shear_table_wl_ns['deg_sep'], bins_deg)
dig_mpc_bins_ns = np.digitize(shear_table_wl_ns['mpc_sep'], bins_mpc)

shear_diff = 0.02

Definitions of tangential and cross shear used:
$$ \gamma_t = -\gamma_1\cos(2\phi)-\gamma_2\sin(2\phi) $$
$$ \gamma_\times = \gamma_1\sin(2\phi)-\gamma_2\cos(2\phi) $$

$\phi$ : Polar Angle of galaxy relative to BCG 

### Individual Phi

This section will attempt to measure $\gamma_t$ and $\gamma_\times$ by:
- Calculate individual $\phi$ for each galaxy
- Calculate $\gamma_t$ and $\gamma_\times$ for each galaxy
- Calculate the response R from all galaxies, no binning
- Apply R to averaged / binned $\gamma_t,\gamma_\times$

The error for R is based on a test case within `ngmix` [here](https://github.com/esheldon/ngmix/blob/d08b471f4c4d5887df9f9f2551efaf8f2e226150/examples/metacal/metadetect.py#L101)

In [ ]:
p1_mean = shear_table_wl_1p['wmom_g_1'].mean()
m1_mean = shear_table_wl_1m['wmom_g_1'].mean()
p2_mean = shear_table_wl_2p['wmom_g_2'].mean()
m2_mean = shear_table_wl_2m['wmom_g_2'].mean()

r_matrix = [[0, 0],[0, 0]]

r_matrix[0][0] = (p1_mean - m1_mean) / shear_diff
# r_matrix[0][1] = (p2_mean - m2_mean) / shear_diff # ignore?
# r_matrix[1][0] = (p1_mean - m1_mean) / shear_diff
r_matrix[1][1] = (p2_mean - m2_mean) / shear_diff

r_matrix_inv = inv(r_matrix)

# calculate R error
p1_err = np.std(shear_table_wl_1p['wmom_g_1']) / np.sqrt(len(shear_table_wl_1p['wmom_g_1']))
m1_err = np.std(shear_table_wl_1m['wmom_g_1']) / np.sqrt(len(shear_table_wl_1m['wmom_g_1']))
p2_err = np.std(shear_table_wl_2p['wmom_g_2']) / np.sqrt(len(shear_table_wl_2p['wmom_g_2']))
m2_err = np.std(shear_table_wl_2m['wmom_g_2']) / np.sqrt(len(shear_table_wl_2m['wmom_g_2']))

r11_err = np.sqrt(p1_err**2 + m1_err**2)
r22_err = np.sqrt(p2_err**2 + m2_err**2)

print("R11, R22: ", r_matrix[0][0], r_matrix[1][1])
print("R11_err, R22_err: ", r11_err, r22_err)
print("Difference of R11 and R22: ", np.abs(r_matrix[0][0]-r_matrix[1][1]))

In [ ]:
# bin tangential and cross shears by radial bins
tan_cross_shears = np.zeros((len(bins_mpc)-1, 2)) # binned tangential and cross shear
mean_mpc = []
tan_errs = []
cross_errs = []

for i in range(0, len(bins_mpc)-1):
    bin_filt_ns = dig_mpc_bins_ns == i+1

    # print number of galaxies in each bin
    print("Rows in bin ", i, " :", len(shear_table_wl_ns['shear_t'][bin_filt_ns]))

    # calulcate mean t and x shears
    mean_g_t = shear_table_wl_ns['shear_t'][bin_filt_ns].mean() # mean g1
    mean_g_x = shear_table_wl_ns['shear_x'][bin_filt_ns].mean() # mean g2

    # calculate errors, 95% confidence interval
    g_t_err = stats.bootstrap([shear_table_wl_ns['shear_t'][bin_filt_ns]], np.mean).confidence_interval
    g_x_err = stats.bootstrap([shear_table_wl_ns['shear_x'][bin_filt_ns]], np.mean).confidence_interval

    # apply calibration 
    shear_cal = r_matrix_inv.dot([mean_g_t, mean_g_x])
    shear_cal_err_upper = r_matrix_inv.dot([g_t_err.high, g_x_err.high])
    shear_cal_err_lower = r_matrix_inv.dot([g_t_err.low, g_x_err.low])

    tan_err = [shear_cal_err_lower[0], shear_cal_err_upper[0]]
    cross_err = [shear_cal_err_lower[1], shear_cal_err_upper[1]]

    tan_cross_shears[i] = shear_cal
    tan_errs.append(tan_err)
    cross_errs.append(cross_err)

    # get the mean distance from BCG
    mean_deg_sep = shear_table_wl_ns['deg_sep'][bin_filt_ns].mean()
    mean_mpc_sep = cosmo.eval_da(0.22) * mean_deg_sep * np.pi/180
    mean_mpc.append(mean_mpc_sep)

tan_errs = np.array(tan_errs)
cross_errs = np.array(cross_errs)

#### Getting NFW Model with CLMM Package

In [ ]:
galcat = GCData()
galcat['ra'] = shear_table_wl_ns['ra']
galcat['dec'] = shear_table_wl_ns['dec']
galcat['e1'] = shear_table_wl_ns['wmom_g_1'] * 2
galcat['e2'] = shear_table_wl_ns['wmom_g_2'] * 2

galcat['z'] = np.zeros(len(shear_table_wl_ns)) # CLMM needs a redshift column for the source, even if not used

cluster_id = "Abell 360"
gc_object1 = clmm.GalaxyCluster(cluster_id, ra_bcg, dec_bcg, 0.22, galcat, coordinate_system='euclidean')

gc_object1.compute_galaxy_weights(
        shape_component1="e1",
        shape_component2="e2",
        use_shape_error=False, # individual shape errors are not yet available for Metadetect
        use_shape_noise=True,
        weight_name="w_ls",
        cosmo=cosmo,
        add=True,
    )

moo = clmm.Modeling(massdef="mean", delta_mdef=200, halo_profile_model="nfw")

moo.set_cosmo(cosmo)
moo.set_concentration(4)
# moo.set_mass(1.0e15)
moo.set_mass(4.0e14)

z_cl = gc_object1.z

# source properties
# assume sources redshift following a the DESC SRD distribution. This will need updating.

z_distrib_func = utils.redshift_distributions.desc_srd  

# Compute first beta (e.g. eq(6) of WtGIII paper)
beta_kwargs = {
    "z_cl": z_cl,
    "z_inf": 10.0,
    "cosmo": cosmo,
    "z_distrib_func": z_distrib_func,
}
beta_s_mean = utils.compute_beta_s_mean_from_distribution(**beta_kwargs)
beta_s_square_mean = utils.compute_beta_s_square_mean_from_distribution(**beta_kwargs)

rproj = np.logspace(np.log10(0.3),np.log10(7.), 100)

gt_z = moo.eval_reduced_tangential_shear(
    rproj, z_cl, [beta_s_mean, beta_s_square_mean], z_src_info="beta", approx="order2"
)

In [ ]:
fig, axes = plt.subplots(1,1, figsize=(8,6))

point_size = 40
point_alpha = 1

# g1 calibrated
axes.scatter(mean_mpc, tan_cross_shears[:,0],
             marker='.', s=point_size, label='tangential shear')
axes.plot(mean_mpc, tan_cross_shears[:,0], '-o')
axes.vlines(mean_mpc, tan_errs[:,0], tan_errs[:,1], colors='blue')

cross_axis = np.add(mean_mpc, 0.06) # add offset to visually differentiate

axes.scatter(cross_axis, tan_cross_shears[:,1],
             marker='.', s=point_size, label='cross shear')
axes.plot(cross_axis, tan_cross_shears[:,1], '-o')
axes.vlines(cross_axis, cross_errs[:,0], cross_errs[:,1], colors='orange')

plt.axhline(0.0, color='k', ls=':')

plt.plot(rproj, gt_z, label='NFW (model, not fit), M500c=4e14 Msun, c=4, n(z)=SRD', ls=':')

plt.xscale('log')
plt.axhline(0.0, color='k', ls=':')
plt.ylim([-0.07,0.09])
plt.xlim([0.3,7])
axes.set_xlabel('R [Mpc]')
axes.set_ylabel("reduced shear")
axes.legend(loc=1)

plt.suptitle("")
# plt.savefig('image_outputs/shear-final.png', bbox_inches='tight')
plt.show()

# Validation

## Check g1/g2 and PSF ellipticities

Check that the PSF and object ellipticities average to ~0

In [ ]:
# print mean values
print("Mean of psfrec_g_1: ", shear_table_wl_ns['psfrec_g_1'].median())
print("Mean of psfrec_g_2: ", shear_table_wl_ns['psfrec_g_2'].median())
print("Mean of wmom_g_1: ", shear_table_wl_ns['wmom_g_1'].median())
print("Mean of wmom_g_2: ", shear_table_wl_ns['wmom_g_2'].median())

In [ ]:
n_bins = 50

fig, axs = plt.subplots(1, 4, sharey=True, tight_layout=True)

axs[0].hist(shear_table_wl_ns['psfrec_g_1'], bins=n_bins)
axs[0].set_title('psfrec_g_1')
axs[1].hist(shear_table_wl_ns['psfrec_g_2'], bins=n_bins)
axs[1].set_title('psfrec_g_2')
axs[2].hist(shear_table_wl_ns['wmom_g_1'], bins=n_bins)
axs[2].set_title('wmom_g_1')
axs[3].hist(shear_table_wl_ns['wmom_g_2'], bins=n_bins)
axs[3].set_title('wmom_g_2')

fig.suptitle("Distribution of ellipticities for PSF and WMOM")

plt.show()

In [ ]:
n_bins = 50

fig, axs = plt.subplots(1, 4, sharey=True, tight_layout=True)

axs[0].hist(shear_table_wl_ns['psfrec_g_1'], bins=n_bins)
axs[0].set_yscale('log')
axs[0].set_title('psfrec_g_1')
axs[1].hist(shear_table_wl_ns['psfrec_g_2'], bins=n_bins)
axs[1].set_title('psfrec_g_2')
axs[1].set_yscale('log')
axs[2].hist(shear_table_wl_ns['wmom_g_1'], bins=n_bins)
axs[2].set_title('wmom_g_1')
axs[2].set_yscale('log')
axs[3].hist(shear_table_wl_ns['wmom_g_2'], bins=n_bins)
axs[3].set_title('wmom_g_2')
axs[3].set_yscale('log')

fig.suptitle("Distribution of ellipticities for PSF and WMOM")

plt.show()

## Cross-correlation with PSF statistics

Based on the DP0.2 tutorial notebook: https://github.com/lsst/tutorial-notebooks/blob/main/DP0.2/12b_PSF_Science_Demo.ipynb

#### Purely in Metadetect (not recommended)

In [ ]:
obj_ra = shear_table_wl_ns['ra']
obj_dec = shear_table_wl_ns['dec']

In [ ]:
psfrec_e = np.sqrt(shear_table_wl_ns['psfrec_g_1']**2 + shear_table_wl_ns['psfrec_g_1']**2) # original PSf
wmom_psf_e = np.sqrt(shear_table_wl_ns['wmom_psf_g_1']**2 + shear_table_wl_ns['wmom_psf_g_1']**2) # reconvolved PSF
obj_e = np.sqrt(shear_table_wl_ns['wmom_g_1']**2 + shear_table_wl_ns['wmom_g_2']**2)

In [ ]:
cat = treecorr.Catalog(ra=obj_ra, dec=obj_dec,
                       k=obj_e - np.mean(obj_e),
                       ra_units='deg', dec_units='deg')

kk_config = {'max_sep': .06, 'min_sep': .0001, 'nbins': 12}

In [ ]:
kk = treecorr.KKCorrelation(kk_config)
kk.process(cat)

In [ ]:
xi = kk.xi
bins = kk.rnom

In [ ]:
_, (ax1, ax2) = plt.subplots(1, 2, figsize=(13, 4.5),
                             gridspec_kw={'wspace': .3})

ax1.set_title(f'PSF ellipticity')
scatter_plot = ax1.scatter(obj_ra, obj_dec, c=obj_e,
                           s=1, cmap='cividis')
ax1.set_xlabel('RA [deg]')
ax1.set_ylabel('DEC [deg]')
plt.colorbar(scatter_plot, ax=ax1, label='PSF ellipticity')

ax2.set_title(f'PSF ellipticity correlation.')
ax2.plot(np.degrees(bins), xi*1e4, 'o-', color='darkblue')
ax2.axhline(0, linestyle='--', color='lightgrey')
ax2.set_xscale('log')
ax2.set_ylabel(r'$\xi \times 10^{4}$', labelpad=10)
ax2.set_xlabel(r'$\theta$ [degree]')

plt.show()

### Look at PSF residuals

Get the stars from the star-galaxy cuts, though some galaxies may contaminate the star sample.

In [ ]:
shear_table_rs_cuts_ns = shear_table_rs_cuts[shear_table_rs_cuts['shear_type']=='ns']

# cut to find stars
star_cut = shear_table_rs_cuts_ns['wmom_T_ratio']<1.05
star_cut &= shear_table_rs_cuts_ns['wmom_s2n']>10
star_cut &= shear_table_rs_cuts_ns['wmom_T']<20
star_cut &= (shear_table_rs_cuts_ns['wmom_color_mag_g-r']).abs()<5
star_cut &= (shear_table_rs_cuts_ns['wmom_color_mag_r-i']).abs()<5
# junk cuts
star_cut &= shear_table_rs_cuts_ns['wmom_T'] < (0.47 - 2.0*shear_table_rs_cuts_ns['wmom_T_err'])
star_cut &= (shear_table_rs_cuts_ns['wmom_T'] * shear_table_rs_cuts_ns['wmom_T_err']) < 0.0055

stars = shear_table_rs_cuts_ns[star_cut]

In [ ]:
psf_df = pd.DataFrame()
psf_g1_g, psf_g2_g, psf_T_g = [], [], []
psf_g1_r, psf_g2_r, psf_T_r = [], [], []
psf_g1_i, psf_g2_i, psf_T_i = [], [], []
psf_tract, psf_patch, psf_cell_x, psf_cell_y = [], [], [], []

for tract, patch in tract_patch_list:
    for band in 'gri':
        coadd = butler.get('deepCoaddCell', 
                         collections = [cell_collection, cell_collection_g], 
                         instrument = 'LSSTComCam', 
                         skymap = 'lsst_cells_v1', 
                         tract = tract, 
                         patch = patch,
                         band = band,)

        pixel_scale = coadd.wcs.getPixelScale().asArcseconds()

        cell_list = list(coadd.cells.keys()) # skips indices that are empty
    
        # for each cell in cell_list:
        for index, cell_index in enumerate(cell_list):
    
            cell = coadd.cells[cell_index]

            psf_im = cell.psf_image
            psf_kernel = afwMath.FixedKernel(psf_im)
            psf = meas.KernelPsf(psf_kernel)
            shape = psf.computeShape(psf_im.getBBox().getCenter())
            i_xx, i_yy, i_xy = shape.getIxx(), shape.getIyy(), shape.getIxy()
            q = Quadrupole(i_xx, i_yy, i_xy)
            s = SeparableDistortionTraceRadius(q)
            e1, e2 = s.getE1(), s.getE2()

            sigma = shape.getDeterminantRadius() * pixel_scale
            
            psf_T = 2 * sigma**2

            if band=='g':
                psf_g1_g.append(e1)
                psf_g2_g.append(e2)
                psf_T_g.append(psf_T)
                psf_tract.append(tract)
                psf_patch.append(patch)
                psf_cell_x.append(cell_index.x)
                psf_cell_y.append(cell_index.y)
            if band=='r':
                psf_g1_r.append(e1)
                psf_g2_r.append(e2)
                psf_T_r.append(psf_T)
            if band=='i':
                psf_g1_i.append(e1)
                psf_g2_i.append(e2)
                psf_T_i.append(psf_T)
    
        del coadd
        gc.collect()

# take a simple average across the three bands
# Metadetect weights based on inverse variance but wanted to keep this computationally simple
psf_df["psf_g1"] = np.add(np.add(psf_g1_g, psf_g1_r), psf_g1_i) / 3
psf_df["psf_g2"] = np.add(np.add(psf_g2_g, psf_g2_r), psf_g2_i) / 3
psf_df["psf_g"] = np.sqrt(psf_df["psf_g1"]**2 + psf_df["psf_g2"]**2)
psf_df["psf_T"] = np.add(np.add(psf_T_g, psf_T_r), psf_T_i) / 3
psf_df["tract"], psf_df["patch"], psf_df["cell_x"], psf_df["cell_y"] = psf_tract, psf_patch, psf_cell_x, psf_cell_y

In [ ]:
residual_e1 = []
residual_e2 = []
residual_e = []
residual_T = []

# find what cell the star is in and calculate the residual PSF
# location of the star in the cell does not matter, since the PSF is constant throughout the cell
for i, star_row in stars.iterrows():

    in_cell = psf_df['tract'] == star_row['tract']
    in_cell &= (psf_df['patch'] % 10) == star_row['patch_x']
    in_cell &= ((psf_df['patch'] - (psf_df['patch'] % 10))//10) == star_row['patch_y']
    in_cell &= psf_df['cell_x'] == star_row['cell_x']
    in_cell &= psf_df['cell_y'] == star_row['cell_y']
    
    psf_row = psf_df[in_cell]

    star_e = np.sqrt(star_row['psfrec_g_1']**2 + star_row['psfrec_g_2']**2)
        
    residual_e1.append((star_row['psfrec_g_1'] - psf_row['psf_g1']).item())
    residual_e2.append((star_row['psfrec_g_2'] - psf_row['psf_g2']).item())
    residual_e.append((star_e - psf_row["psf_g"]).item())
    residual_T.append((star_row['psfrec_T'] - psf_row['psf_T']).item())

In [ ]:
cat = treecorr.Catalog(ra=stars['ra'], dec=stars['dec'],
                       k=residual_T - np.mean(residual_T),
                       ra_units='deg', dec_units='deg')

kk_config = {'max_sep': .06, 'min_sep': .0001, 'nbins': 12}

In [ ]:
kk = treecorr.KKCorrelation(kk_config)
kk.process(cat)

In [ ]:
xi = kk.xi
bins = kk.rnom

In [ ]:
_, (ax1, ax2) = plt.subplots(1, 2, figsize=(13, 4.5),
                             gridspec_kw={'wspace': .3})

ax1.set_title(f'PSF ellipticity')
scatter_plot = ax1.scatter(stars['ra'], stars['dec'], c=residual_T,
                           s=1, cmap='cividis')
ax1.set_xlabel('RA [deg]')
ax1.set_ylabel('DEC [deg]')
plt.colorbar(scatter_plot, ax=ax1, label='PSF ellipticity')

ax2.set_title(f'PSF ellipticity correlation.')
ax2.plot(np.degrees(bins), xi*1e4, 'o-', color='darkblue')
ax2.axhline(0, linestyle='--', color='lightgrey')
ax2.set_xscale('log')
ax2.set_ylabel(r'$\xi \times 10^{4}$', labelpad=10)
ax2.set_xlabel(r'$\theta$ [degree]')

plt.show()

### Look at reserved stars in object table

Note that these reserved stars are not measured on the cell-based coadds. This is meant as an estimate.

In [ ]:
obj_collection = 'LSSTComCam/runs/DRP/DP1/w_2025_17/DM-50530'

In [ ]:
obj_table_list = []

for tract, patch in tract_patch_list:
    
    dataId = {'tract': tract, 'patch' : patch , 'skymap':'lsst_cells_v1'}
    obj_cat = butler.get('object_patch', dataId=dataId, collections=obj_collection)
    obj_table = obj_cat.to_pandas()

    # Only include stars "reserved" to check the PSF modeling
    filt = obj_cat['detect_isPrimary'] == True
    filt &= obj_cat['refExtendedness'] == 0.0
    filt &= obj_cat['i_calib_psf_reserved'] == True
    filt &= obj_cat['i_pixelFlags_inexact_psfCenter']==False

    obj_table_list.append(obj_table[filt])

obj_table = pd.concat(obj_table_list)

# remove unused tables to clear up memory
del obj_table_list
gc.collect()

In [ ]:
# method find the pixel indices that overlap the sky projection of the cell area
def update_reserved_stars_cell(cell, cell_index, wcs, star_table, star_cells):
    cell_bbox = cell.inner.bbox
    begin_coord = wcs.pixelToSky(cell_bbox.beginX, cell_bbox.beginY)
    end_coord = wcs.pixelToSky(cell_bbox.endX, cell_bbox.endY)
    
    if begin_coord.getRa() < end_coord.getRa():
        ra1 = begin_coord.getRa().asDegrees()
        ra2 = end_coord.getRa().asDegrees()
    else:
        ra1 = end_coord.getRa().asDegrees()
        ra2 = begin_coord.getRa().asDegrees()
    
    if begin_coord.getDec() < end_coord.getDec():
        dec1 = begin_coord.getDec().asDegrees()
        dec2 = end_coord.getDec().asDegrees()
    else:
        dec1 = end_coord.getDec().asDegrees()
        dec2 = begin_coord.getDec().asDegrees()

    # check which stars fall within the cell
    for i, star_row in star_table.iterrows():

        in_ra = (star_row['coord_ra'] > ra1) & (star_row['coord_ra'] < ra2)
        in_dec = (star_row['coord_dec'] > dec1) & (star_row['coord_dec'] < dec2)

        if in_ra & in_dec:
            # append cell info to the star table
            star_cells.append(cell_index)

In [ ]:
# collect cell information in only the i-band
psf_df = pd.DataFrame()
psf_g1_i, psf_g2_i, psf_T_i = [], [], []
psf_tract, psf_patch, psf_cell_x, psf_cell_y = [], [], [], []
star_cells = []

overlap_patches_9 = [9, 19, 29, 39, 49, 59, 69, 79, 89, 99]
overlap_patches_1 = [1, 11, 21, 31, 41, 51, 61, 71, 81, 91]

for tract, patch in tract_patch_list:
    
    coadd = butler.get('deepCoaddCell', 
                     collections = [cell_collection, cell_collection_g], 
                     instrument = 'LSSTComCam', 
                     skymap = 'lsst_cells_v1', 
                     tract = tract, 
                     patch = patch,
                     band = 'i',)

    wcs = coadd.wcs

    pixel_scale = coadd.wcs.getPixelScale().asArcseconds()

    cell_list = list(coadd.cells.keys()) # skips indices that are empty

    # for each cell in cell_list:
    for index, cell_index in enumerate(cell_list):

        # skip duplicates due to patch overlap
        if cell_index.x == 0 or cell_index.x == 21 or cell_index.y == 0 or cell_index.y == 21:
            continue

        
        # skip duplicates due to tract overlap
        if tract==10464 and patch in overlap_patches_9 and cell_index.x == 20:
            continue

        if tract==10463 and patch in overlap_patches_1 and cell_index.x == 1:
            continue
            
        cell = coadd.cells[cell_index]

        psf_im = cell.psf_image
        psf_kernel = afwMath.FixedKernel(psf_im)
        psf = meas.KernelPsf(psf_kernel)
        shape = psf.computeShape(psf_im.getBBox().getCenter())
        i_xx, i_yy, i_xy = shape.getIxx(), shape.getIyy(), shape.getIxy()
        q = Quadrupole(i_xx, i_yy, i_xy)
        s = SeparableDistortionTraceRadius(q)
        e1, e2 = s.getE1(), s.getE2()

        sigma = shape.getDeterminantRadius() * pixel_scale
        
        psf_T = 2 * sigma**2

        psf_tract.append(tract)
        psf_patch.append(patch)
        psf_cell_x.append(cell_index.x)
        psf_cell_y.append(cell_index.y)

        psf_g1_i.append(e1)
        psf_g2_i.append(e2)
        psf_T_i.append(psf_T)

        update_reserved_stars_cell(cell, cell_index, wcs, obj_table, star_cells)
    
    del coadd
    gc.collect()

# take a simple average across the three bands
# Metadetect weights based on inverse variance but wanted to keep this computationally simple
psf_df["psf_g1"] = np.array(psf_g1_i)
psf_df["psf_g2"] = np.array(psf_g2_i)
psf_df["psf_g"] = np.sqrt(psf_df["psf_g1"]**2 + psf_df["psf_g2"]**2)
psf_df["psf_T"] = np.array(psf_T_i)
psf_df["tract"], psf_df["patch"], psf_df["cell_x"], psf_df["cell_y"] = psf_tract, psf_patch, psf_cell_x, psf_cell_y

In [ ]:
def get_star_ellip(catalog):
    star_mxx = catalog['i_ixx']
    star_myy = catalog['i_iyy']
    star_mxy = catalog['i_ixy']
    return (star_mxx - star_myy) / (star_mxx + star_myy), 2. * star_mxy / (star_mxx + star_myy)

In [ ]:
residual_e1 = []
residual_e2 = []
residual_e = []
residual_T = []

# location of the star in the cell does not matter, since the PSF is constant throughout the cell
row_count = 0
for i, star_row in obj_table.iterrows():

    star_e1, star_e2 = get_star_ellip(star_row)
    star_e = np.sqrt(star_e1**2 + star_e2**2)

    in_cell = psf_df['tract'] == star_row['tract']
    in_cell &= psf_df['patch'] == star_row['patch']
    in_cell &= psf_df['cell_x'] == star_cells[row_count].x
    in_cell &= psf_df['cell_y'] == star_cells[row_count].y
    
    psf_row = psf_df[in_cell]

    residual_e1.append((star_e1 - psf_row['psf_g1']).item())
    residual_e2.append((star_e2 - psf_row['psf_g2']).item())
    residual_e.append((star_e - psf_row["psf_g"]).item())
    # residual_T.append((star_row['psfrec_T'] - psf_row['psf_T']).item())

    row_count += 1

In [ ]:
kk_config = {'max_sep': .06, 'min_sep': .0001, 'nbins': 12}


# e1
cat_e1 = treecorr.Catalog(ra=obj_table['coord_ra'], dec=obj_table['coord_dec'],
                       k=residual_e1 - np.mean(residual_e1),
                       ra_units='deg', dec_units='deg')

kk_e1 = treecorr.KKCorrelation(kk_config)
kk_e1.process(cat_e1)
xi_e1 = kk_e1.xi
xi_e1_err = kk_e1.varxi
bins_e1 = kk_e1.rnom

# e2
cat_e2 = treecorr.Catalog(ra=obj_table['coord_ra'], dec=obj_table['coord_dec'],
                       k=residual_e2 - np.mean(residual_e2),
                       ra_units='deg', dec_units='deg')

kk_e2 = treecorr.KKCorrelation(kk_config)
kk_e2.process(cat_e2)
xi_e2 = kk_e2.xi
xi_e2_err = kk_e2.varxi
bins_e2 = kk_e2.rnom

# |e|
cat_e = treecorr.Catalog(ra=obj_table['coord_ra'], dec=obj_table['coord_dec'],
                       k=residual_e - np.mean(residual_e),
                       ra_units='deg', dec_units='deg')

kk_e = treecorr.KKCorrelation(kk_config)
kk_e.process(cat_e)
xi_e = kk_e.xi
xi_e_err = kk_e.varxi
bins_e = kk_e.rnom

# collect errors
xi_e1_err_plot = [xi_e1*1e4 - xi_e1_err*1e7, xi_e1*1e4 + xi_e1_err*1e7]
xi_e2_err_plot = [xi_e2*1e4 - xi_e2_err*1e7, xi_e2*1e4 + xi_e2_err*1e7]
xi_e_err_plot = [xi_e*1e4 - xi_e_err*1e7, xi_e*1e4 + xi_e_err*1e7]

In [ ]:
fig, axes = plt.subplots(2, 3, figsize=(17, 9), gridspec_kw={'wspace': 0.35, 'hspace': 0.25})

# e1
axes[0][0].set_title(f'e1 PSF residuals (star minus model)')
scatter_plot = axes[0][0].scatter(obj_table['coord_ra'], obj_table['coord_dec'], c=residual_e1, s=10)
axes[0][0].set_xlabel('RA [deg]')
axes[0][0].set_ylabel('DEC [deg]')
plt.colorbar(scatter_plot, ax=axes[0][0], label='e1 residual')

axes[1][0].set_title(f'e1 PSF residuals correlation')
axes[1][0].plot(np.degrees(bins_e1), xi_e1*1e4, 'o-', color='darkblue')
axes[1][0].vlines(np.degrees(bins_e1), xi_e1_err_plot[0], xi_e1_err_plot[1], colors='darkblue')
axes[1][0].axhline(0, linestyle='--', color='lightgrey')

axes[1][0].set_xscale('log')
axes[1][0].set_ylabel(r'$\xi \times 10^{4}$', labelpad=10)
axes[1][0].set_xlabel(r'$\theta$ [degree]')

# e2
axes[0][1].set_title(f'e2 PSF residuals (star minus model)')
scatter_plot = axes[0][1].scatter(obj_table['coord_ra'], obj_table['coord_dec'], c=residual_e2, s=10)
axes[0][1].set_xlabel('RA [deg]')
axes[0][1].set_ylabel('DEC [deg]')
plt.colorbar(scatter_plot, ax=axes[0][1], label='e2 residual')

axes[1][1].set_title(f'e2 PSF residuals correlation')
axes[1][1].plot(np.degrees(bins_e2), xi_e2*1e4, 'o-', color='darkblue')
axes[1][1].vlines(np.degrees(bins_e2), xi_e2_err_plot[0], xi_e2_err_plot[1], colors='darkblue')
axes[1][1].axhline(0, linestyle='--', color='lightgrey')
axes[1][1].set_xscale('log')
axes[1][1].set_ylabel(r'$\xi \times 10^{4}$', labelpad=10)
axes[1][1].set_xlabel(r'$\theta$ [degree]')

# e
axes[0][2].set_title(f'|e| PSF residuals (star minus model)')
scatter_plot = axes[0][2].scatter(obj_table['coord_ra'], obj_table['coord_dec'], c=residual_e, s=10)
axes[0][2].set_xlabel('RA [deg]')
axes[0][2].set_ylabel('DEC [deg]')
plt.colorbar(scatter_plot, ax=axes[0][2], label='e residual')

axes[1][2].set_title(f'|e| PSF residuals correlation')
axes[1][2].plot(np.degrees(bins_e), xi_e*1e4, 'o-', color='darkblue')
axes[1][2].vlines(np.degrees(bins_e), xi_e_err_plot[0], xi_e_err_plot[1], colors='darkblue')
axes[1][2].axhline(0, linestyle='--', color='lightgrey')
axes[1][2].set_xscale('log')
axes[1][2].set_ylabel(r'$\xi \times 10^{4}$', labelpad=10)
axes[1][2].set_xlabel(r'$\theta$ [degree]')

plt.suptitle("PSF Residual Correlations", y=0.95, size=15)

# plt.savefig('image_outputs/psf_correlations.png', bbox_inches='tight')
plt.show()

## Plot `wmom_T_ratio` vs S/N

In [ ]:
fig, axes = plt.subplots(1,2, figsize=(12,5))

axes[0].scatter(np.log(shear_table_rs_cuts['wmom_s2n']), shear_table_rs_cuts['wmom_T_ratio'], 
           marker='.', s=0.5, alpha=point_alpha)
axes[0].set_xlabel('log(S/N)')
axes[0].set_ylabel('wmom_T_ratio')
axes[0].hlines(1.1, 2.9, 11.1, color='red')
axes[0].set_title('Prior to wmom_T_ratio > 1.1 cut')

axes[1].scatter(np.log(shear_table_wl_ns['wmom_s2n']), shear_table_wl_ns['wmom_T_ratio'], 
           marker='.', s=0.5, alpha=point_alpha)
axes[1].set_xlabel('log(S/N)')
axes[1].set_ylabel('wmom_T_ratio')
axes[1].set_title('After wmom_T_ratio > 1.1 cut')

# plt.savefig('image_outputs/obj_T_vs_s2n.png', bbox_inches='tight')
plt.show()

## Plot g-r vs r-i

In [ ]:
fig, axes = plt.subplots(1,3, figsize=(16,5))

shear_table_md_cuts_ns = shear_table_md_cuts[shear_table_md_cuts['shear_type']=='ns']
shear_table_rs_cuts_ns = shear_table_rs_cuts[shear_table_rs_cuts['shear_type']=='ns']
# shear_table_wl_ns = shear_table_wl[shear_table_wl['shear_type']=='ns']

axes[0].scatter(shear_table_md_cuts_ns['wmom_color_mag_g-r'], shear_table_md_cuts_ns['wmom_color_mag_r-i'], 
           marker='.', s=0.3, alpha=point_alpha)
axes[0].set_xlabel('g-r')
axes[0].set_ylabel('r-i')
axes[0].set_title('After MD cuts')

axes[1].scatter(shear_table_rs_cuts_ns['wmom_color_mag_g-r'], shear_table_rs_cuts_ns['wmom_color_mag_r-i'], 
           marker='.', s=0.3, alpha=point_alpha)
axes[1].set_xlabel('g-r')
axes[1].set_ylabel('r-i')
axes[1].set_title('After RS cuts')

axes[2].scatter(shear_table_wl_ns['wmom_color_mag_g-r'], shear_table_wl_ns['wmom_color_mag_r-i'], 
           marker='.', s=0.3, alpha=point_alpha)
axes[2].set_xlabel('g-r')
axes[2].set_ylabel('r-i')
axes[2].set_title('After quality cuts')

plt.show()

## Plot object density as a function of every cut

In [ ]:
obj_density = shear_table_rs_cuts.copy()
obj_density = obj_density[obj_density['shear_type']=='ns']

In [ ]:
obj_density_t_ratio = obj_density[obj_density['wmom_T_ratio']>1.1]
obj_density_s2n = obj_density[obj_density['wmom_s2n']>10]
obj_density_t = obj_density[obj_density['wmom_T']<20]
obj_density_mfrac = obj_density[obj_density['mfrac']<0.1]
obj_density_gmag = obj_density[obj_density['wmom_band_mag_g']<26.5]
obj_density_rmag = obj_density[obj_density['wmom_band_mag_r']<26.5]
obj_density_imag = obj_density[obj_density['wmom_band_mag_i']<24.7]
obj_density_gr = obj_density[(obj_density['wmom_color_mag_g-r']).abs()<5]
obj_density_ri = obj_density[(obj_density['wmom_color_mag_r-i']).abs()<5]
obj_density_junk1 = obj_density[obj_density['wmom_T'] < (0.425 - 2.0*obj_density['wmom_T_err'])]
obj_density_junk2 = obj_density[(obj_density['wmom_T'] * obj_density['wmom_T_err']) < 0.006]
# obj_density_bright # placeholder

In [ ]:
fig, axes = plt.subplots(1,3, figsize=(19,6), sharex=True, sharey=True)

axes[0].scatter(shear_table_dup['ra'][shear_table_dup['shear_type']=='ns'], shear_table_dup['dec'][shear_table_dup['shear_type']=='ns'], marker='.', s=0.2) 
axes[0].set_title("No cuts")
axes[0].set_xlabel("ra")
axes[0].set_ylabel("dec")

axes[1].scatter(obj_density['ra'], obj_density['dec'], marker='.', s=0.2) 
axes[1].set_title("Only Red Sequence Cuts")
axes[1].set_xlabel("ra")
axes[1].set_ylabel("dec")

axes[2].scatter(shear_table_wl_ns['ra'], shear_table_wl_ns['dec'], marker='.', s=0.2) 
axes[2].set_title("All cuts")
axes[2].set_xlabel("ra")
axes[2].set_ylabel("dec")
axes[2].set_xlim(ra_bcg-0.7, ra_bcg+0.7)
axes[2].set_ylim(dec_bcg-0.7, dec_bcg+0.7)

for ax in axes.reshape(-1):
    ax.scatter([ra_bcg], [dec_bcg], marker='+', s=100, color='orange')

plt.suptitle("Object Distribution", size=16, y=0.98)
# plt.savefig('image_outputs/object-distribution-before-after.png', bbox_inches='tight')
plt.show()

In [ ]:
fig, axes = plt.subplots(4,4, figsize=(15,16), sharex=True, sharey=True)
fig.tight_layout(h_pad=2.5, rect=[0, 0.03, 1, 0.95])

axes[0][0].scatter(obj_density['ra'], obj_density['dec'], marker='.', s=0.2) 
axes[0][0].set_title("No cuts")
axes[0][0].set_ylabel("dec")

axes[0][1].scatter(obj_density_t_ratio['ra'], obj_density_t_ratio['dec'], marker='.', s=0.2) 
axes[0][1].set_title("T_ratio")

axes[0][2].scatter(obj_density_s2n['ra'], obj_density_s2n['dec'], marker='.', s=0.2) 
axes[0][2].set_title("S/N")

axes[0][3].scatter(obj_density_t['ra'], obj_density_t['dec'], marker='.', s=0.2) 
axes[0][3].set_title("T")

axes[1][0].scatter(obj_density_mfrac['ra'], obj_density_mfrac['dec'], marker='.', s=0.2) 
axes[1][0].set_title("mfrac")
axes[1][0].set_ylabel("dec")

axes[1][1].scatter(obj_density_gmag['ra'], obj_density_gmag['dec'], marker='.', s=0.2) 
axes[1][1].set_title("mag_g")

axes[1][2].scatter(obj_density_rmag['ra'], obj_density_rmag['dec'], marker='.', s=0.2) 
axes[1][2].set_title("mag_g")

axes[1][3].scatter(obj_density_imag['ra'], obj_density_imag['dec'], marker='.', s=0.2) 
axes[1][3].set_title("mag_i")

axes[2][0].scatter(obj_density_gr['ra'], obj_density_gr['dec'], marker='.', s=0.2) 
axes[2][0].set_title("g-r color cut")
axes[2][0].set_ylabel("dec")

axes[2][1].scatter(obj_density_ri['ra'], obj_density_ri['dec'], marker='.', s=0.2) 
axes[2][1].set_title("r-i color cut")

axes[2][2].scatter(obj_density_junk1['ra'], obj_density_junk1['dec'], marker='.', s=0.2) 
axes[2][2].set_title("T vs. T_err cut")

axes[2][3].scatter(obj_density_junk2['ra'], obj_density_junk2['dec'], marker='.', s=0.2) 
axes[2][3].set_title("T x T_err cut")

axes[3][0].scatter(shear_table_wl_ns['ra'], shear_table_wl_ns['dec'], marker='.', s=0.2) 
axes[3][0].set_title("All cuts")
axes[3][0].set_ylabel("dec")
axes[3][0].set_xlabel("ra")

axes[3][1].set_xlabel("ra")
axes[3][2].set_xlabel("ra")
axes[3][3].set_xlabel("ra")

for ax in axes.reshape(-1):
    ax.scatter([ra_bcg], [dec_bcg], marker='+', s=100, color='orange')

plt.suptitle("Object Distribution Per Cut", size=16, y=0.98)
# plt.savefig('image_outputs/object-distribution-cuts.png', bbox_inches='tight')
plt.show()

## Junk Detections

In [ ]:
x_1 = np.linspace(0.0, 0.0175, num=10)
line_y = 0.425 - 2.0*x_1

fig, axes = plt.subplots(1,2, figsize=(14,6), sharex=True, sharey=True)
# fig.tight_layout(h_pad=2.5, rect=[0, 0.03, 1, 0.95])

axes[0].plot(x_1, line_y, color='red', linewidth=0.7)
axes[0].scatter(shear_table_rs_cuts['wmom_T_err'][shear_table_rs_cuts['shear_type']=='ns'], 
                shear_table_rs_cuts['wmom_T'][shear_table_rs_cuts['shear_type']=='ns'], marker='.', s=0.3)
axes[0].set_title("After RS cuts cuts")
axes[0].set_xlabel("wmom_T_err")
axes[0].set_ylabel("wmom_T")

axes[1].plot(x_1, line_y, color='red', linewidth=0.7)
axes[1].scatter(shear_table_wl['wmom_T_err'][shear_table_wl['shear_type']=='ns'], 
                shear_table_wl['wmom_T'][shear_table_wl['shear_type']=='ns'], marker='.', s=0.3)
axes[1].set_title("After all cuts")
axes[1].set_xlabel("wmom_T_err")
axes[1].set_ylabel("wmom_T")

plt.suptitle("T versus T_err", size=16, y=0.98)
# plt.savefig('image_outputs/junk1.png', bbox_inches='tight')
plt.show()

In [ ]:
x_1 = np.linspace(0.0, 0.0175, num=10)
line_y = 0.425 - 2.0*x_1

fig, axes = plt.subplots(1,2, figsize=(14,6), sharex=True, sharey=True)
# fig.tight_layout(h_pad=2.5, rect=[0, 0.03, 1, 0.95])

axes[0].hist(shear_table_rs_cuts['wmom_T']*shear_table_rs_cuts['wmom_T_err'], bins=75)
axes[0].vlines(0.006, 0, 9000, color='red')
axes[0].set_title("T x T_err (after RS cuts)")
axes[0].set_xlabel("wmom_T x wmom_T_err")
axes[0].set_yscale("log")

axes[1].hist(shear_table_wl['wmom_T']*shear_table_wl['wmom_T_err'], bins=75)
axes[1].vlines(0.006, 0, 2200, color='red')
axes[1].set_title("T x T_err (after all cuts)")
axes[1].set_xlabel("wmom_T x wmom_T_err")

plt.suptitle("T times T_err", size=16, y=0.98)
# plt.savefig('image_outputs/junk2.png', bbox_inches='tight')
plt.show()

### Find Junk Objects

In [ ]:
t_err_line_filt = shear_table_rs_cuts['wmom_T'] > (0.425 - 2.0*shear_table_rs_cuts['wmom_T_err'])
t_t_err_filt = (shear_table_rs_cuts['wmom_T'] * shear_table_rs_cuts['wmom_T_err']) > 0.006
junk_obj_1 = shear_table_rs_cuts[t_err_line_filt]
junk_obj_2 = shear_table_rs_cuts[t_t_err_filt]

### Display Junk Object

Code primarily from Robert Lupton's demo notebook: https://github.com/RobertLuptonTheGood/notebooks/blob/2eeee8b9fe35077387485e488c965f1ea3d39418/Demos/Colour%20Images.ipynb

In [ ]:
junk_obj_2[['tract', 'patch_y', 'patch_x', 'ra', 'dec']][100:150]

In [ ]:
tract = 10463
patch = 41

bands = "gri"
images = {}

# get x, y coordinates of object in patch
filt_loc = junk_obj_2['tract'] == tract
filt_loc &= junk_obj_2['patch_y'] == 4
filt_loc &= junk_obj_2['patch_x'] == 1
# filt_loc &= junk_obj_2['shear_type'] == 'ns'
# filt_loc = shear_table_wl['tract'] == tract
# filt_loc &= shear_table_wl['patch_y'] == 9
# filt_loc &= shear_table_wl['patch_x'] == 1

xys_firefly = []
xys = []

for b in bands:

    coadd = butler.get("deepCoaddCell", 
                    tract=tract, 
                    patch=patch, 
                    band=b, 
                    skymap='lsst_cells_v1', 
                    collections=[cell_collection, cell_collection_g])

    if b=='g':
        bbox = coadd.outer_bbox
        for i, row in junk_obj_2[filt_loc].iterrows():
            radecs = [row['ra'], row['dec']]
            raDec = geom.SpherePoint(row['ra']*geom.degrees, row['dec']*geom.degrees)
            xy_test = geom.PointI(coadd.wcs.skyToPixel(raDec))
            xys_firefly.append(xy_test)
            xy = xy_test - bbox.getBegin()
            xys.append(xy)
            
    images[b] = coadd.stitch().asMaskedImage()
    
    del coadd
    gc.collect()

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(12, 12))

for coord in xys:
    circle = plt.Circle((coord.getX(), coord.getY()), 50, color='cyan', fill=False)
    ax.add_patch(circle)

rgb = afwRgb.makeRGB(images['i'], images['r'], images['g'], 9, 20, Q=2)
afwRgb.displayRGB(rgb)
plt.show()

### Look at objects in weak lensing sample

In [ ]:
tract = 10463
patch = 62

bands = "gri"
images = {}

# get x, y coordinates of object in patch
filt_loc = shear_table_wl_ns['tract'] == tract
filt_loc &= shear_table_wl_ns['patch_y'] == 6
filt_loc &= shear_table_wl_ns['patch_x'] == 2

xys_firefly = []
xys = []

for b in bands:

    coadd = butler.get("deepCoaddCell", 
                    tract=tract, 
                    patch=patch, 
                    band=b, 
                    skymap='lsst_cells_v1', 
                    collections=[cell_collection, cell_collection_g])

    if b=='g':
        bbox = coadd.outer_bbox
        for i, row in shear_table_wl_ns[filt_loc].iterrows():
            radecs = [row['ra'], row['dec']]
            raDec = geom.SpherePoint(row['ra']*geom.degrees, row['dec']*geom.degrees)
            xy_test = geom.PointI(coadd.wcs.skyToPixel(raDec))
            xys_firefly.append(xy_test)
            xy = xy_test - bbox.getBegin()
            xys.append(xy)
            
    images[b] = coadd.stitch().asMaskedImage()
    
    del coadd
    gc.collect()

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(12, 12))

for coord in xys:
    circle = plt.Circle((coord.getX(), coord.getY()), 50, color='cyan', fill=False)
    ax.add_patch(circle)

rgb = afwRgb.makeRGB(images['i'], images['r'], images['g'], 4, 15, Q=2)
afwRgb.displayRGB(rgb)
plt.show()

### Try with firefly

In [ ]:
# import lsst.afw.display as afwDisplay
# afwDisplay.setDefaultBackend('firefly')
# display1 = afwDisplay.Display(frame=1)
# display1.getClient().show_lab_tab

# display1.mtv(coadd.stitch().image)
# with display1.Buffering():
#     for coord in xys:
#         display1.dot('o', coord.getX(), coord.getY(), size=50, ctype='orange')

# # display1.erase()

## Plot distribution of cell properties in A360 region

Each run of the `get_cell_inputs` function should only take 3-4 minutes. If it's longer, try again later.

In [ ]:
from lsst.sphgeom import Box, HealpixPixelization
import healsparse as hsp
import healpy as hp
from hpgeom import hpgeom
import skyproj

In [ ]:
# define healpix parameters
nside_coverage = 2**8
nside_sparse = 2**14

pixelization = HealpixPixelization(hp.nside2order(nside_sparse))

In [ ]:
# method find the pixel indices that overlap the sky projection of the cell area
def get_cell_pixels(cell, wcs):
    cell_bbox = cell.inner.bbox
    begin_coord = wcs.pixelToSky(cell_bbox.beginX, cell_bbox.beginY)
    end_coord = wcs.pixelToSky(cell_bbox.endX, cell_bbox.endY)
    
    if begin_coord.getRa() < end_coord.getRa():
        ra1 = begin_coord.getRa().asDegrees()
        ra2 = end_coord.getRa().asDegrees()
    else:
        ra1 = end_coord.getRa().asDegrees()
        ra2 = begin_coord.getRa().asDegrees()
    
    if begin_coord.getDec() < end_coord.getDec():
        dec1 = begin_coord.getDec().asDegrees()
        dec2 = end_coord.getDec().asDegrees()
    else:
        dec1 = end_coord.getDec().asDegrees()
        dec2 = begin_coord.getDec().asDegrees()

    indices = hpgeom.query_box(nside=nside_sparse, a0=ra1, a1=ra2, b0=dec1, b1=dec2)
    
    return indices

In [ ]:
# return PSF e magnitude of the cell
def get_psf_e(cell, wcs):
    # get psf image of cell
    psf_im = cell.psf_image
    
    psf_kernel = afwMath.FixedKernel(psf_im)
    psf = meas.KernelPsf(psf_kernel)
    shape = psf.computeShape(psf_im.getBBox().getCenter())
    
    trace_radius = shape.getTraceRadius()
    i_xx, i_yy, i_xy = shape.getIxx(), shape.getIyy(), shape.getIxy()
    
    q = Quadrupole(i_xx, i_yy, i_xy)
    s = SeparableDistortionTraceRadius(q)
    
    e1, e2 = s.getE1(), s.getE2()
    e = np.sqrt(e1**2 + e2**2)
    
    return e

In [ ]:
# returns the limiting PSF magnitude of each pixel, given the cell object and statistics settings
def get_mag_lim(cell, statsCtrl, zp=31.4, num_sigma=5):
    
    # get psf area of cell
    psf_im = cell.psf_image.array
    numer = np.square(np.sum(psf_im))
    # numer = 1 # I've seen this as well, but said to be prone to changing if stamp size changes
    denom = np.sum(np.square(psf_im))
    psf_area = numer/denom

    # get total cell weight
    accTask = AssembleCellCoaddTask()
    mask_im = cell.inner.asMaskedImage()
    computed_weight = accTask._compute_weight(mask_im, statsCtrl)
    # the computed weight will be the same value for each pixel
    total_weight = computed_weight * mask_im.getDimensions()[0] * mask_im.getDimensions()[1]
    
    maglim = zp-2.5*np.log10(num_sigma*np.sqrt(psf_area/total_weight))
    
    return maglim

In [ ]:
def get_cell_inputs(cell_collection, tract_patch_list, band):

    cell_df = pd.DataFrame()
    cell_ra = []
    cell_dec = []
    pixel_indices = []
    inputs_list = []
    mag_lim_list = []
    psf_e_list = []

    segs = [] # collection of lines to plot patch outlines

    for tract, patch in tract_patch_list:
    
        coadd = butler.get('deepCoaddCell', 
                         collections = cell_collection, 
                         instrument = 'LSSTComCam', 
                         skymap = 'lsst_cells_v1', 
                         tract = tract, 
                         patch = patch,
                         band = band,)
        # define a wcs from the given coadd
        wcs = coadd.wcs

        # get coadd outline
        coadd_corners = coadd.inner_bbox.getCorners()
    
        for index, corner in enumerate(coadd_corners):
            corner_coord_start = wcs.pixelToSky(corner.getX(), corner.getY())
            if index < 3:
                corner_coord_end = wcs.pixelToSky(coadd_corners[index+1].getX(), coadd_corners[index+1].getY())
            else:
                corner_coord_end = wcs.pixelToSky(coadd_corners[0].getX(), coadd_corners[0].getY())
    
            start_ra = corner_coord_start[0].asDegrees()
            start_dec = corner_coord_start[1].asDegrees()
    
            end_ra = corner_coord_end[0].asDegrees()
            end_dec = corner_coord_end[1].asDegrees()
    
            segs.append(((start_ra, start_dec), (end_ra, end_dec)))
        
        cell_list = list(coadd.cells.keys()) # skips indices that are empty

        # for each cell in cell_list:
        for index, cell_index in enumerate(cell_list):
    
            cell = coadd.cells[cell_index]
    
            # get cell coordinates for removing duplicates
            cell_center = cell.inner.bbox.getCenter()
            cell_center_coord = wcs.pixelToSky(cell_center)
            cell_ra.append(cell_center_coord.getRa().asDegrees())
            cell_dec.append(cell_center_coord.getDec().asDegrees())
        
            pixel_indices.append(get_cell_pixels(cell, wcs))
    
            inputs_list.append(cell.visit_count)
            mag_lim_list.append(get_mag_lim(cell, statsCtrl))
            psf_e_list.append(get_psf_e(cell, wcs))
    
        del coadd
        gc.collect()

    cell_df["ra"] = cell_ra
    cell_df["dec"] = cell_dec
    cell_df["pixels"] = pixel_indices
    cell_df["inputs"] = inputs_list
    cell_df["mag_lim"] = mag_lim_list
    cell_df["psf_e"] = psf_e_list

    return cell_df, segs

#### g-band

In [ ]:
cell_df_g, segs_g = get_cell_inputs(cell_collection_g, tract_patch_list, 'g')

In [ ]:
# remove duplicate cells from overlapping patches
cell_df_g = cell_df_g.drop_duplicates(subset=['ra', 'dec'])

In [ ]:
pixel_df_g = cell_df_g.explode('pixels').reset_index(drop=True)
pixel_df_g = pixel_df_g.drop_duplicates(subset=["pixels"]) 
pixel_df_g = pixel_df_g.dropna(subset=['pixels'])

pixels_g = pixel_df_g["pixels"].to_numpy()
pixel_input_g = pixel_df_g["inputs"].to_numpy()
pixel_mag_lim_g = pixel_df_g["mag_lim"].to_numpy()
pixel_psf_e_g = pixel_df_g["psf_e"].to_numpy()

In [ ]:
hsp_map_input_g = hsp.HealSparseMap.make_empty(nside_coverage, nside_sparse, np.int64)
hsp_map_input_g.update_values_pix(np.array(pixels_g, dtype=np.int64), np.array(pixel_input_g))

hsp_map_mag_lim_g = hsp.HealSparseMap.make_empty(nside_coverage, nside_sparse, np.float64)
hsp_map_mag_lim_g.update_values_pix(np.array(pixels_g, dtype=np.int64), np.array(pixel_mag_lim_g))

hsp_map_psf_e_g = hsp.HealSparseMap.make_empty(nside_coverage, nside_sparse, np.float64)
hsp_map_psf_e_g.update_values_pix(np.array(pixels_g, dtype=np.int64), np.array(pixel_psf_e_g))

#### r-band

In [ ]:
cell_df_r, segs_r = get_cell_inputs(cell_collection, tract_patch_list, 'r')

In [ ]:
# remove duplicate cells from overlapping patches
cell_df_r = cell_df_r.drop_duplicates(subset=['ra', 'dec'])

In [ ]:
pixel_df_r = cell_df_r.explode('pixels').reset_index(drop=True)
pixel_df_r = pixel_df_r.drop_duplicates(subset=["pixels"]) 
pixel_df_r = pixel_df_r.dropna(subset=['pixels'])

pixels_r = pixel_df_r["pixels"].to_numpy()
pixel_input_r = pixel_df_r["inputs"].to_numpy()
pixel_mag_lim_r = pixel_df_r["mag_lim"].to_numpy()
pixel_psf_e_r = pixel_df_r["psf_e"].to_numpy()

In [ ]:
hsp_map_input_r = hsp.HealSparseMap.make_empty(nside_coverage, nside_sparse, np.int64)
hsp_map_input_r.update_values_pix(np.array(pixels_r, dtype=np.int64), np.array(pixel_input_r))

hsp_map_mag_lim_r = hsp.HealSparseMap.make_empty(nside_coverage, nside_sparse, np.float64)
hsp_map_mag_lim_r.update_values_pix(np.array(pixels_r, dtype=np.int64), np.array(pixel_mag_lim_r))

hsp_map_psf_e_r = hsp.HealSparseMap.make_empty(nside_coverage, nside_sparse, np.float64)
hsp_map_psf_e_r.update_values_pix(np.array(pixels_r, dtype=np.int64), np.array(pixel_psf_e_r))

#### i-band

In [ ]:
cell_df_i, segs_i = get_cell_inputs(cell_collection, tract_patch_list, 'i')

In [ ]:
# remove duplicate cells from overlapping patches
cell_df_i = cell_df_i.drop_duplicates(subset=['ra', 'dec'])

In [ ]:
pixel_df_i = cell_df_i.explode('pixels').reset_index(drop=True)
pixel_df_i = pixel_df_i.drop_duplicates(subset=["pixels"]) 
pixel_df_i = pixel_df_i.dropna(subset=['pixels'])

pixels_i = pixel_df_i["pixels"].to_numpy()
pixel_input_i = pixel_df_i["inputs"].to_numpy()
pixel_mag_lim_i = pixel_df_i["mag_lim"].to_numpy()
pixel_psf_e_i = pixel_df_i["psf_e"].to_numpy()

In [ ]:
hsp_map_input_i = hsp.HealSparseMap.make_empty(nside_coverage, nside_sparse, np.int64)
hsp_map_input_i.update_values_pix(np.array(pixels_i, dtype=np.int64), np.array(pixel_input_i))

hsp_map_mag_lim_i = hsp.HealSparseMap.make_empty(nside_coverage, nside_sparse, np.float64)
hsp_map_mag_lim_i.update_values_pix(np.array(pixels_i, dtype=np.int64), np.array(pixel_mag_lim_i))

hsp_map_psf_e_i = hsp.HealSparseMap.make_empty(nside_coverage, nside_sparse, np.float64)
hsp_map_psf_e_i.update_values_pix(np.array(pixels_i, dtype=np.int64), np.array(pixel_psf_e_i))

#### Input Distribution Plot

The three missing patches were due to pipeline failures, though are not included within the 0.5 degree radius of the BCG

In [ ]:
fig, ax = plt.subplots(1, 3, figsize=(18, 5.3))
sp_g = skyproj.GnomonicSkyproj(ax=ax[0], lon_0=37.862, lat_0=6.98, min_lon_ticklabel_delta=0.2)
sp_g.draw_hspmap(hsp_map_input_g)
sp_g.ax.set_xlabel("RA (deg)", fontsize=10,)
sp_g.ax.set_ylabel("DEC (deg)", fontsize=10,)
sp_g.draw_colorbar(shrink=0.8, label='Number of input warps per cell')
sp_g.ax.set_title("g-band", pad=25)
for seg in segs_g:
    sp_g.ax.plot([seg[0][0], seg[1][0]], [seg[0][1], seg[1][1]], 'r-', alpha=0.6)

sp_r = skyproj.GnomonicSkyproj(ax=ax[1], lon_0=37.862, lat_0=6.98, min_lon_ticklabel_delta=0.2)
sp_r.draw_hspmap(hsp_map_input_r)
sp_r.ax.set_xlabel("RA (deg)", fontsize=10,)
sp_r.ax.set_ylabel("DEC (deg)", fontsize=10,)
sp_r.draw_colorbar(shrink=0.8, label='Number of input warps per cell')
sp_r.ax.set_title("r-band", pad=25)
for seg in segs_r:
    sp_r.ax.plot([seg[0][0], seg[1][0]], [seg[0][1], seg[1][1]], 'r-', alpha=0.6)
    
sp_i = skyproj.GnomonicSkyproj(ax=ax[2], lon_0=37.862, lat_0=6.98, min_lon_ticklabel_delta=0.2)
sp_i.draw_hspmap(hsp_map_input_i)
sp_i.ax.set_xlabel("RA (deg)", fontsize=10,)
sp_i.ax.set_ylabel("DEC (deg)", fontsize=10,)
sp_i.draw_colorbar(shrink=0.8, label='Number of input warps per cell')
sp_i.ax.set_title("i-band", pad=25)
for seg in segs_i:
    sp_i.ax.plot([seg[0][0], seg[1][0]], [seg[0][1], seg[1][1]], 'r-', alpha=0.6)

plt.suptitle("Input Image Distribution of the A360 Region", size=15)

# plt.savefig('image_outputs/3_band_image_distribution.png', bbox_inches='tight')
plt.show()

#### Limiting Magnitude Distribution Plot

In [ ]:
fig, ax = plt.subplots(1, 3, figsize=(19.5, 5.5))
sp_g = skyproj.GnomonicSkyproj(ax=ax[0], lon_0=37.862, lat_0=6.98, min_lon_ticklabel_delta=0.2)
sp_g.draw_hspmap(hsp_map_mag_lim_g)
sp_g.ax.set_xlabel("RA (deg)", fontsize=10,)
sp_g.ax.set_ylabel("DEC (deg)", fontsize=10,)
sp_g.draw_colorbar(shrink=0.9, label='5-sigma Limiting Magnitude')
sp_g.ax.set_title("g-band", pad=25)
for seg in segs_g:
    sp_g.ax.plot([seg[0][0], seg[1][0]], [seg[0][1], seg[1][1]], 'r-', alpha=0.6)

sp_r = skyproj.GnomonicSkyproj(ax=ax[1], lon_0=37.862, lat_0=6.98, min_lon_ticklabel_delta=0.2)
sp_r.draw_hspmap(hsp_map_mag_lim_r)
sp_r.ax.set_xlabel("RA (deg)", fontsize=10,)
sp_r.ax.set_ylabel("DEC (deg)", fontsize=10,)
sp_r.draw_colorbar(shrink=0.9, label='5-sigma Limiting Magnitude')
sp_r.ax.set_title("r-band", pad=25)
for seg in segs_r:
    sp_r.ax.plot([seg[0][0], seg[1][0]], [seg[0][1], seg[1][1]], 'r-', alpha=0.6)
    
sp_i = skyproj.GnomonicSkyproj(ax=ax[2], lon_0=37.862, lat_0=6.98, min_lon_ticklabel_delta=0.2)
sp_i.draw_hspmap(hsp_map_mag_lim_i)
sp_i.ax.set_xlabel("RA (deg)", fontsize=10,)
sp_i.ax.set_ylabel("DEC (deg)", fontsize=10,)
sp_i.draw_colorbar(shrink=0.9, label='5-sigma Limiting Magnitude')
sp_i.ax.set_title("i-band", pad=25)
for seg in segs_i:
    sp_i.ax.plot([seg[0][0], seg[1][0]], [seg[0][1], seg[1][1]], 'r-', alpha=0.6)

plt.suptitle("5-sigma Limiting Magnitude Distribution of the A360 Region", size=15)

# plt.savefig('image_outputs/3_band_mag_lim_distribution.png', bbox_inches='tight')
plt.show()

There may be some noise missing in the covariance due to the warping process. These plots should be taken with a grain of salt.

#### PSF Ellipticity Distribution Plot

In [ ]:
# mean PSF ellipticies for each band
print("Mean PSF ellipticity in g-band: ", pixel_psf_e_g.mean())
print("Mean PSF ellipticity in r-band: ", pixel_psf_e_r.mean())
print("Mean PSF ellipticity in i-band: ", pixel_psf_e_i.mean())
print()
# median PSF ellipticies for each band
print("Median PSF ellipticity in g-band: ", np.median(pixel_psf_e_g))
print("Median PSF ellipticity in r-band: ", np.median(pixel_psf_e_r))
print("Median PSF ellipticity in i-band: ", np.median(pixel_psf_e_i))

In [ ]:
fig, ax = plt.subplots(1, 3, figsize=(19.5, 5.5))
sp_g = skyproj.GnomonicSkyproj(ax=ax[0], lon_0=37.862, lat_0=6.98, min_lon_ticklabel_delta=0.2)
sp_g.draw_hspmap(hsp_map_psf_e_g)
sp_g.ax.circle(ra_bcg, dec_bcg, 0.5, color='cyan')
sp_g.ax.set_xlabel("RA (deg)", fontsize=10,)
sp_g.ax.set_ylabel("DEC (deg)", fontsize=10,)
sp_g.draw_colorbar(shrink=0.9, label='PSF Ellipticity')
sp_g.ax.set_title("g-band", pad=25)
for seg in segs_g:
    sp_g.ax.plot([seg[0][0], seg[1][0]], [seg[0][1], seg[1][1]], 'r-', alpha=0.6)

sp_r = skyproj.GnomonicSkyproj(ax=ax[1], lon_0=37.862, lat_0=6.98, min_lon_ticklabel_delta=0.2)
sp_r.draw_hspmap(hsp_map_psf_e_r)
sp_r.ax.circle(ra_bcg, dec_bcg, 0.5, color='cyan')
sp_r.ax.set_xlabel("RA (deg)", fontsize=10,)
sp_r.ax.set_ylabel("DEC (deg)", fontsize=10,)
sp_r.draw_colorbar(shrink=0.9, label='PSF Ellipticity')
sp_r.ax.set_title("r-band", pad=25)
for seg in segs_r:
    sp_r.ax.plot([seg[0][0], seg[1][0]], [seg[0][1], seg[1][1]], 'r-', alpha=0.6)
    
sp_i = skyproj.GnomonicSkyproj(ax=ax[2], lon_0=37.862, lat_0=6.98, min_lon_ticklabel_delta=0.2)
sp_i.draw_hspmap(hsp_map_psf_e_i)
sp_g.ax.circle(ra_bcg, dec_bcg, 0.5, color='cyan')
sp_i.ax.set_xlabel("RA (deg)", fontsize=10,)
sp_i.ax.set_ylabel("DEC (deg)", fontsize=10,)
sp_i.draw_colorbar(shrink=0.9, label='PSF Ellipticity')
sp_i.ax.set_title("i-band", pad=25)
for seg in segs_i:
    sp_i.ax.plot([seg[0][0], seg[1][0]], [seg[0][1], seg[1][1]], 'r-', alpha=0.6)

plt.suptitle("PSF Ellipticity Distribution of the A360 Region", size=15)

# plt.savefig('image_outputs/3_band_psf_e_distribution.png', bbox_inches='tight')
plt.show()

## Investigate Patch Failures

In [ ]:
meta_log = 0
meta_log_ids = []
for ref in butler.registry.queryDatasets('metadetectionShear_log', collections=collection):
    meta_log += 1
    meta_log_ids.append(ref.dataId)

    patch_log = butler.get('metadetectionShear_log', 
                     collections = collection, 
                     instrument = 'LSSTComCam', 
                     skymap = 'lsst_cells_v1', 
                     tract = ref.dataId['tract'], 
                     patch = ref.dataId['patch'],
                     band = 'i')
    
    patch_error = [log['message'] for log in patch_log.model_dump() if log['levelname'] == 'ERROR']
    if len(patch_error) > 0:
        print(patch_error)

## Look at shear profile with no calibration

In [ ]:
# bin tangential and cross shears by radial bins
tan_cross_shears = np.zeros((len(bins_mpc)-1, 2)) # binned tangential and cross shear
mean_mpc = []
tan_errs = []
cross_errs = []

for i in range(0, len(bins_mpc)-1):
    bin_filt_ns = dig_mpc_bins_ns == i+1

    # print number of galaxies in each bin
    print("Rows in bin ", i, " :", len(shear_table_wl_ns['shear_t'][bin_filt_ns]))

    # calulcate mean t and x shears
    mean_g_t = shear_table_wl_ns['shear_t'][bin_filt_ns].mean() # mean g1
    mean_g_x = shear_table_wl_ns['shear_x'][bin_filt_ns].mean() # mean g2

    # calculate errors, 95% confidence interval
    g_t_err = stats.bootstrap([shear_table_wl_ns['shear_t'][bin_filt_ns]], np.mean).confidence_interval
    g_x_err = stats.bootstrap([shear_table_wl_ns['shear_x'][bin_filt_ns]], np.mean).confidence_interval

    tan_err = [g_t_err.low, g_t_err.high]
    cross_err = [g_x_err.low, g_x_err.high]
    
    # apply calibration 
    gs = [mean_g_t, mean_g_x]

    tan_cross_shears[i] = gs
    tan_errs.append(tan_err)
    cross_errs.append(cross_err)

    # get the mean distance from BCG
    mean_deg_sep = shear_table_wl_ns['deg_sep'][bin_filt_ns].mean()
    mean_mpc_sep = cosmo.eval_da(0.22) * mean_deg_sep * np.pi/180
    mean_mpc.append(mean_mpc_sep)

tan_errs = np.array(tan_errs)
cross_errs = np.array(cross_errs)

In [ ]:
galcat = GCData()
galcat['ra'] = shear_table_wl_ns['ra']
galcat['dec'] = shear_table_wl_ns['dec']
galcat['e1'] = shear_table_wl_ns['wmom_g_1'] * 2
galcat['e2'] = shear_table_wl_ns['wmom_g_2'] * 2

galcat['z'] = np.zeros(len(shear_table_wl_ns)) # CLMM needs a redshift column for the source, even if not used

cluster_id = "Abell 360"
gc_object1 = clmm.GalaxyCluster(cluster_id, ra_bcg, dec_bcg, 0.22, galcat, coordinate_system='euclidean')

gc_object1.compute_galaxy_weights(
        shape_component1="e1",
        shape_component2="e2",
        use_shape_error=False,
        use_shape_noise=True,
        weight_name="w_ls",
        cosmo=cosmo,
        add=True,
    )

moo = clmm.Modeling(massdef="mean", delta_mdef=200, halo_profile_model="nfw")

moo.set_cosmo(cosmo)
moo.set_concentration(4)
moo.set_mass(4.0e14)

z_cl = gc_object1.z

# source properties
# assume sources redshift following a the DESC SRD distribution. This will need updating.

z_distrib_func = utils.redshift_distributions.desc_srd  

# Compute first beta (e.g. eq(6) of WtGIII paper)
beta_kwargs = {
    "z_cl": z_cl,
    "z_inf": 10.0,
    "cosmo": cosmo,
    "z_distrib_func": z_distrib_func,
}
beta_s_mean = utils.compute_beta_s_mean_from_distribution(**beta_kwargs)
beta_s_square_mean = utils.compute_beta_s_square_mean_from_distribution(**beta_kwargs)

rproj = np.logspace(np.log10(0.3),np.log10(7.), 100)

gt_z = moo.eval_reduced_tangential_shear(
    rproj, z_cl, [beta_s_mean, beta_s_square_mean], z_src_info="beta", approx="order2"
)

In [ ]:
fig, axes = plt.subplots(1,1, figsize=(8,6))

point_size = 40
point_alpha = 1

# g1 calibrated
axes.scatter(mean_mpc, tan_cross_shears[:,0],
             marker='.', s=point_size, label='tangential shear')
axes.plot(mean_mpc, tan_cross_shears[:,0], '-o')
axes.vlines(mean_mpc, tan_errs[:,0], tan_errs[:,1], colors='blue')

cross_axis = np.add(mean_mpc, 0.05) # add offset to visually differentiate

axes.scatter(cross_axis, tan_cross_shears[:,1],
             marker='.', s=point_size, label='cross shear')
axes.plot(cross_axis, tan_cross_shears[:,1], '-o')
axes.vlines(cross_axis, cross_errs[:,0], cross_errs[:,1], colors='orange')

plt.axhline(0.0, color='k', ls=':')

plt.plot(rproj, gt_z, label='NFW (model, not fit), M500c=4e14 Msun, c=4, n(z)=SRD', ls=':')

plt.xscale('log')
plt.axhline(0.0, color='k', ls=':')
# plt.ylim([-0.03,0.08])
plt.ylim([-0.07,0.09])
plt.xlim([0.3,7])
#plt.yscale('log')
axes.set_xlabel('R [Mpc]')
axes.set_ylabel("reduced shear")
axes.legend(loc=1)

# plt.savefig('image_outputs/shear-no-cal.png', bbox_inches='tight')
plt.show()

## Look at shear profile with R calculated with galaxies beyond 0.5 degree radius

In [ ]:
collection_all = 'u/mgorsuch/metadetect/comcam_Rubin_SV_38_7_3bands/20250621T235449Z'

In [ ]:
datasetRefs_shear_all = []
overlap_patches_left = [0, 10, 20, 30, 40, 50, 60, 70, 80, 90, 100]
overlap_patches_down = [0, 1, 2, 3, 4, 5, 6, 7, 8, 9]
tract_patch_list_all = []

for ref in butler.registry.queryDatasets('ShearObject', collections=collection_all):

    if ref.dataId['tract'] == 10705 and ref.dataId['patch'] in overlap_patches_down:
        continue

    if ref.dataId['tract'] == 10704 and ref.dataId['patch'] in overlap_patches_down:
        continue

    if ref.dataId['tract'] == 10704 and ref.dataId['patch'] in overlap_patches_left:
        continue

    if ref.dataId['tract'] == 10463 and ref.dataId['patch'] in overlap_patches_left:
        continue
    
    datasetRefs_shear_all.append(butler.query_datasets('ShearObject', 
                                                     collections=collection_all,
                                                     skymap = 'lsst_cells_v1',
                                                     tract=ref.dataId['tract'],
                                                     patch=ref.dataId['patch']))

    tract_patch_list_all.append([ref.dataId['tract'], ref.dataId['patch']])

In [ ]:
shear_table_list = []

for i, ref in enumerate(datasetRefs_shear_all):
    shear_data_patch = butler.get(ref[0])
    shear_table_patch = shear_data_patch.to_pandas()
    shear_table_list.append(shear_table_patch)

shear_table_all = pd.concat(shear_table_list)

# remove unused tables to clear up memory
del shear_table_list
gc.collect()

In [ ]:
# remove objects in outer ring of cells in each patch since patch overlap is two cells
# TO-DO: exempt rows that don't overlap with other patches, e.g. patches on the edge of the field
shear_table_all = shear_table_all[shear_table_all['cell_x']!=0]
shear_table_all = shear_table_all[shear_table_all['cell_x']!=21]
shear_table_all = shear_table_all[shear_table_all['cell_y']!=0]
shear_table_all = shear_table_all[shear_table_all['cell_y']!=21]
print("Number of rows after removing most duplicate cells: ", len(shear_table_all))

# some additional tract/patch overlap appears to have a 4 cell overlap
filt1 = shear_table_all['tract'] == 10464
filt1 &= shear_table_all['patch_x'] == 9
filt1 &= shear_table_all['cell_x'] == 20
shear_table_all = shear_table_all[np.invert(filt1)]

filt2 = shear_table_all['tract'] == 10463
filt2 &= shear_table_all['patch_x'] == 1
filt2 &= shear_table_all['cell_x'] == 1
shear_table_all = shear_table_all[np.invert(filt2)]

filt3 = shear_table_all['tract'] == 10464
filt3 &= shear_table_all['patch_y'] == 9
filt3 &= shear_table_all['cell_y'] == 20
shear_table_all = shear_table_all[np.invert(filt3)]

filt4 = shear_table_all['tract'] == 10463
filt4 &= shear_table_all['patch_y'] == 9
filt4 &= shear_table_all['cell_y'] == 20
shear_table_all = shear_table_all[np.invert(filt4)]

### 
filt5 = shear_table_all['tract'] == 10705
filt5 &= shear_table_all['patch_x'] == 9
filt5 &= shear_table_all['cell_x'] == 20
shear_table_all = shear_table_all[np.invert(filt5)]

filt6 = shear_table_all['tract'] == 10704
filt6 &= shear_table_all['patch_x'] == 1
filt6 &= shear_table_all['cell_x'] == 1
shear_table_all = shear_table_all[np.invert(filt6)]

filt7 = shear_table_all['tract'] == 10705
filt7 &= shear_table_all['patch_y'] == 1
filt7 &= shear_table_all['cell_y'] == 1
shear_table_all = shear_table_all[np.invert(filt7)]

filt8 = shear_table_all['tract'] == 10704
filt8 &= shear_table_all['patch_y'] == 1
filt8 &= shear_table_all['cell_y'] == 1
shear_table_all = shear_table_all[np.invert(filt8)]

print("Number of rows after removing patch overlap in 10464: ", len(shear_table_all))

# remove overlapping rows due to patch overlap    
print("Number of rows prior to removing duplicates: ", len(shear_table_all))
shear_table_all = shear_table_all.drop_duplicates(subset=['shear_type', 'ra', 'dec']) # each object will potentially have several sheared images
print("Number of rows after removing duplicates: ", len(shear_table_all))

In [ ]:
# make new columns to convert nJy fluxes to AB magnitudes
t1 = Table.from_pandas(shear_table_all)

t1['wmom_band_mag_g'] = (t1['wmom_band_flux_g']*u.nJy).to(u.ABmag)
t1['wmom_band_mag_r'] = (t1['wmom_band_flux_r']*u.nJy).to(u.ABmag)
t1['wmom_band_mag_i'] = (t1['wmom_band_flux_i']*u.nJy).to(u.ABmag)
t1['wmom_color_mag_g-r'] = t1['wmom_band_mag_g']-t1['wmom_band_mag_r']
t1['wmom_color_mag_g-i'] = t1['wmom_band_mag_g']-t1['wmom_band_mag_i']
t1['wmom_color_mag_r-i'] = t1['wmom_band_mag_r']-t1['wmom_band_mag_i']

shear_table_all = t1.to_pandas()

# Add columns for distance from BCG
c1 = SkyCoord(shear_table_all['ra'].values*u.deg, shear_table_all['dec'].values*u.deg)
c2 = SkyCoord(ra_bcg*u.deg, dec_bcg*u.deg)
sep = c1.separation(c2)

shear_table_all['deg_sep'] = sep.value

shear_table_all['mpc_sep'] = cosmo.eval_da(0.22) * shear_table_all['deg_sep'] * np.pi/180

# polar angle of source galaxy relative to BCG, from -pi to pi
shear_table_all['phi'] = np.arctan2(shear_table_all['dec'] - dec_bcg, (ra_bcg - shear_table_all['ra'])*np.cos(np.deg2rad(dec_bcg)))

# calculate tangential / cross shear components for each galaxy
# note that these shears need to be selected for ONLY the 'ns' (no artificial shear type)
shear_table_all['shear_t'] = -shear_table_all['wmom_g_1'] * np.cos(2*shear_table_all['phi']) \
                                - shear_table_all['wmom_g_2'] * np.sin(2*shear_table_all['phi'])
shear_table_all['shear_x'] = shear_table_all['wmom_g_1'] * np.sin(2*shear_table_all['phi']) \
                                - shear_table_all['wmom_g_2'] * np.cos(2*shear_table_all['phi'])

In [ ]:
# remove overlapping rows due to patch overlap    
print("Number of rows prior to removing duplicates: ", len(shear_table_all))
shear_table_all = shear_table_all.drop_duplicates(subset=['shear_type', 'ra', 'dec']) # each object will potentially have several sheared images
print("Number of rows after removing duplicates: ", len(shear_table_all))

In [ ]:
meta_filter = shear_table_all['wmom_flags']==False
meta_filter &= shear_table_all['psfrec_flags']==False
meta_filter &= shear_table_all['wmom_psf_flags']==False
meta_filter &= shear_table_all['wmom_obj_flags']==False
meta_filter &= shear_table_all['wmom_T_flags']==False
meta_filter &= shear_table_all['wmom_band_flux_flags_r']==False
meta_filter &= shear_table_all['wmom_band_flux_flags_i']==False

shear_table_all = shear_table_all[meta_filter]

print("Number of rows after removing metadetect flags: ", len(shear_table_all))
print("Number of rows in ns after removing metadetect flags: ", len(shear_table_all[shear_table_all['shear_type']=='ns']))

# shear_table_all_wl = shear_table_all[shear_table_all['deg_sep'] < 0.5] # catalog for WL measurements
shear_table_all_wl = shear_table_all # catalog for WL measurements
shear_table_all_wl_ns = shear_table_all_wl[shear_table_all_wl['shear_type'] == 'ns']
print("Number of rows in ns after applying < 0.5 deg from center: ", len(shear_table_all_wl))

rs_hi_ri = r_up_left + r_slope * (shear_table_all_wl['wmom_band_mag_r']-r_left)
rs_low_ri = r_down_left + r_slope * (shear_table_all_wl['wmom_band_mag_r']-r_left)
ri_filt = shear_table_all_wl['wmom_color_mag_r-i'] > rs_low_ri
ri_filt &= shear_table_all_wl['wmom_color_mag_r-i'] < rs_hi_ri
ri_filt &= shear_table_all_wl['wmom_band_mag_r'] < brightness_cut

rs_hi_gi = g_up_left + g_slope * (shear_table_all_wl['wmom_band_mag_g']-g_left)
rs_low_gi = g_down_left + g_slope * (shear_table_all_wl['wmom_band_mag_g']-g_left)
gi_filt = shear_table_all_wl['wmom_color_mag_g-i'] > rs_low_gi
gi_filt &= shear_table_all_wl['wmom_color_mag_g-i'] < rs_hi_gi
gi_filt &= shear_table_all_wl['wmom_band_mag_g'] < brightness_cut

# ns only, for plotting
ri_filt_ns = np.logical_and(ri_filt, shear_table_all_wl['shear_type'] == 'ns')
gi_filt_ns = np.logical_and(gi_filt, shear_table_all_wl['shear_type'] == 'ns')

rs_filt = np.logical_and(ri_filt, gi_filt)
rs_filt_ns = np.logical_and(rs_filt, shear_table_all_wl['shear_type'] == 'ns')

RS_id_list = shear_table_all_wl['id'][rs_filt]
RS_id_list_ns = shear_table_all_wl_ns['id'][rs_filt_ns] # for plotting

# Filter out rows where the 'dataid' column matches any value in RS_id_list
shear_table_all_wl = shear_table_all_wl[~shear_table_all_wl['id'].isin(RS_id_list)]
shear_table_all_wl_ns = shear_table_all_wl_ns[~shear_table_all_wl_ns['id'].isin(RS_id_list_ns)]

print("Number of rows after 0.5 degree cut and RS cuts: ", len(shear_table_all_wl))
print("Number of rows after 0.5 degree cut and RS cuts, ns only: ", len(shear_table_all_wl_ns))

# looser cuts for more objects
loose_cuts = shear_table_all_wl['wmom_T_ratio']>1.1
loose_cuts &= shear_table_all_wl['wmom_s2n']>10
loose_cuts &= shear_table_all_wl['wmom_T']<20
loose_cuts &= shear_table_all_wl['mfrac']<0.1
loose_cuts &= shear_table_all_wl['wmom_band_mag_g']<24.7
loose_cuts &= shear_table_all_wl['wmom_band_mag_r']<24.2
loose_cuts &= shear_table_all_wl['wmom_band_mag_i']<23.6
loose_cuts &= shear_table_all_wl['wmom_band_mag_i']>20.0
loose_cuts &= (shear_table_all_wl['wmom_color_mag_g-r']).abs()<5
loose_cuts &= (shear_table_all_wl['wmom_color_mag_r-i']).abs()<5
# junk cuts
loose_cuts &= shear_table_all_wl['wmom_T'] < (0.425 - 2.0*shear_table_all_wl['wmom_T_err'])
loose_cuts &= (shear_table_all_wl['wmom_T'] * shear_table_all_wl['wmom_T_err']) < 0.006

cut_name, cut_name_string = loose_cuts, "loose_cuts"
print_rows_removed(shear_table_all_wl, cut_name_string)

shear_table_all_wl = shear_table_all_wl[cut_name]
shear_table_all_wl_ns = shear_table_all_wl[shear_table_all_wl['shear_type']=='ns']

print()
print("Number of rows after applying quality cuts: ", len(shear_table_all_wl))

In [ ]:
plt.scatter(shear_table_all_wl_ns['ra'], shear_table_all_wl_ns['dec'], marker='.', s=0.2)
plt.scatter([ra_bcg], [dec_bcg], marker='+', s=100, color='orange')
plt.title("Object Distribution after all cuts")

Lots of gaps. Unsure why some patches were fine before with the smaller collection.

In [ ]:
# Radial binning, either in Mpc or degrees
bins_deg = clmm.make_bins(0.025,0.5,nbins=6, method='evenlog10width')
bins_mpc = cosmo.eval_da(0.22) * bins_deg * np.pi/180

# define distance bins
dig_rad_bins_ns = np.digitize(shear_table_wl_ns['deg_sep'], bins_deg)
dig_mpc_bins_ns = np.digitize(shear_table_wl_ns['mpc_sep'], bins_mpc)

shear_diff = 0.02

In [ ]:
# split catalog by shear type
shear_table_all_all_1p = shear_table_all_wl[shear_table_all_wl['shear_type']=='1p']
shear_table_all_all_1m = shear_table_all_wl[shear_table_all_wl['shear_type']=='1m']
shear_table_all_all_2p = shear_table_all_wl[shear_table_all_wl['shear_type']=='2p']
shear_table_all_all_2m = shear_table_all_wl[shear_table_all_wl['shear_type']=='2m']

In [ ]:
p1_mean = shear_table_all_all_1p['wmom_g_1'].mean()
m1_mean = shear_table_all_all_1m['wmom_g_1'].mean()
p2_mean = shear_table_all_all_2p['wmom_g_2'].mean()
m2_mean = shear_table_all_all_2m['wmom_g_2'].mean()

r_matrix = [[0, 0],[0, 0]]

r_matrix[0][0] = (p1_mean - m1_mean) / shear_diff
# r_matrix[0][1] = (p2_mean - m2_mean) / shear_diff # ignore?
# r_matrix[1][0] = (p1_mean - m1_mean) / shear_diff
r_matrix[1][1] = (p2_mean - m2_mean) / shear_diff

r_matrix_inv = inv(r_matrix)

# calculate R error
p1_err = np.std(shear_table_all_all_1p['wmom_g_1']) / np.sqrt(len(shear_table_all_all_1p['wmom_g_1']))
m1_err = np.std(shear_table_all_all_1m['wmom_g_1']) / np.sqrt(len(shear_table_all_all_1m['wmom_g_1']))
p2_err = np.std(shear_table_all_all_2p['wmom_g_2']) / np.sqrt(len(shear_table_all_all_2p['wmom_g_2']))
m2_err = np.std(shear_table_all_all_2m['wmom_g_2']) / np.sqrt(len(shear_table_all_all_2m['wmom_g_2']))

r11_err = np.sqrt(p1_err**2 + m1_err**2)
r22_err = np.sqrt(p2_err**2 + m2_err**2)

print("R11, R22: ", r_matrix[0][0], r_matrix[1][1])
print("R11_err, R22_err: ", r11_err, r22_err)
print("Difference of R11 and R22: ", np.abs(r_matrix[0][0]-r_matrix[1][1]))

In [ ]:
# bin tangential and cross shears by radial bins
tan_cross_shears = np.zeros((len(bins_mpc)-1, 2)) # binned tangential and cross shear
mean_mpc = []
tan_errs = []
cross_errs = []

for i in range(0, len(bins_mpc)-1):
    bin_filt_ns = dig_mpc_bins_ns == i+1

    # print number of galaxies in each bin
    print("Rows in bin ", i, " :", len(shear_table_wl_ns['shear_t'][bin_filt_ns]))

    # calulcate mean t and x shears
    mean_g_t = shear_table_wl_ns['shear_t'][bin_filt_ns].mean() # mean g1
    mean_g_x = shear_table_wl_ns['shear_x'][bin_filt_ns].mean() # mean g2

    # calculate errors, 95% confidence interval
    g_t_err = stats.bootstrap([shear_table_wl_ns['shear_t'][bin_filt_ns]], np.mean).confidence_interval
    g_x_err = stats.bootstrap([shear_table_wl_ns['shear_x'][bin_filt_ns]], np.mean).confidence_interval

    # apply calibration 
    shear_cal = r_matrix_inv.dot([mean_g_t, mean_g_x])
    shear_cal_err_upper = r_matrix_inv.dot([g_t_err.high, g_x_err.high])
    shear_cal_err_lower = r_matrix_inv.dot([g_t_err.low, g_x_err.low])

    tan_err = [shear_cal_err_lower[0], shear_cal_err_upper[0]]
    cross_err = [shear_cal_err_lower[1], shear_cal_err_upper[1]]

    tan_cross_shears[i] = shear_cal
    tan_errs.append(tan_err)
    cross_errs.append(cross_err)

    # get the mean distance from BCG
    mean_deg_sep = shear_table_wl_ns['deg_sep'][bin_filt_ns].mean()
    mean_mpc_sep = cosmo.eval_da(0.22) * mean_deg_sep * np.pi/180
    mean_mpc.append(mean_mpc_sep)

tan_errs = np.array(tan_errs)
cross_errs = np.array(cross_errs)

In [ ]:
galcat = GCData()
galcat['ra'] = shear_table_wl_ns['ra']
galcat['dec'] = shear_table_wl_ns['dec']
galcat['e1'] = shear_table_wl_ns['wmom_g_1'] * 2
galcat['e2'] = shear_table_wl_ns['wmom_g_2'] * 2

galcat['z'] = np.zeros(len(shear_table_wl_ns)) # CLMM needs a redshift column for the source, even if not used

cluster_id = "Abell 360"
gc_object1 = clmm.GalaxyCluster(cluster_id, ra_bcg, dec_bcg, 0.22, galcat, coordinate_system='euclidean')

gc_object1.compute_galaxy_weights(
        shape_component1="e1",
        shape_component2="e2",
        use_shape_error=False, # individual shape errors are not yet available for Metadetect
        use_shape_noise=True,
        weight_name="w_ls",
        cosmo=cosmo,
        add=True,
    )

moo = clmm.Modeling(massdef="mean", delta_mdef=200, halo_profile_model="nfw")

moo.set_cosmo(cosmo)
moo.set_concentration(4)
# moo.set_mass(1.0e15)
moo.set_mass(4.0e14)

z_cl = gc_object1.z

# source properties
# assume sources redshift following a the DESC SRD distribution. This will need updating.

z_distrib_func = utils.redshift_distributions.desc_srd  

# Compute first beta (e.g. eq(6) of WtGIII paper)
beta_kwargs = {
    "z_cl": z_cl,
    "z_inf": 10.0,
    "cosmo": cosmo,
    "z_distrib_func": z_distrib_func,
}
beta_s_mean = utils.compute_beta_s_mean_from_distribution(**beta_kwargs)
beta_s_square_mean = utils.compute_beta_s_square_mean_from_distribution(**beta_kwargs)

rproj = np.logspace(np.log10(0.3),np.log10(7.), 100)

gt_z = moo.eval_reduced_tangential_shear(
    rproj, z_cl, [beta_s_mean, beta_s_square_mean], z_src_info="beta", approx="order2"
)

In [ ]:
fig, axes = plt.subplots(1,1, figsize=(8,6))

point_size = 40
point_alpha = 1

# g1 calibrated
axes.scatter(mean_mpc, tan_cross_shears[:,0],
             marker='.', s=point_size, label='tangential shear')
axes.plot(mean_mpc, tan_cross_shears[:,0], '-o')
axes.vlines(mean_mpc, tan_errs[:,0], tan_errs[:,1], colors='blue')

cross_axis = np.add(mean_mpc, 0.05) # add offset to visually differentiate

axes.scatter(cross_axis, tan_cross_shears[:,1],
             marker='.', s=point_size, label='cross shear')
axes.plot(cross_axis, tan_cross_shears[:,1], '-o')
axes.vlines(cross_axis, cross_errs[:,0], cross_errs[:,1], colors='orange')

plt.axhline(0.0, color='k', ls=':')

plt.plot(rproj, gt_z, label='NFW (model, not fit), M500c=4e14 Msun, c=4, n(z)=SRD', ls=':')

plt.xscale('log')
plt.axhline(0.0, color='k', ls=':')
# plt.ylim([-0.03,0.08])
plt.ylim([-0.07,0.09])
plt.xlim([0.3,7])
axes.set_xlabel('R [Mpc]')
axes.set_ylabel("reduced shear")
axes.legend(loc=1)

# plt.savefig('image_outputs/shear-all-cal.png', bbox_inches='tight')
plt.show()